In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

#### Crypto Order Book Description 

Crypto PROVIDED order book features/columns (_x represent order book level)

    * midpoint = the midpoint between the best bid and the best ask
    * spread = the difference between the best bid and the best ask

    * bids/asks_distance_x = the distance of bid/ask level x from the midprice in %
                           = (price - midpoint) / midpoint

    * bids/asks_limit/market/cancel_notional_x = volume (= price * quantity) of orders at bid/ask level_x
    * bids/asks_notional_x = (asks_limit_notional_x - asks_market_notional_x - asks_cancel_notional_x)

    Open Items
    * buys = ?
    * sells = ?
    
New DERIVED order book features/columns developed in this notebook/python script

    * bids/ask_price_x = the price at bid/ask level_x
                       = midpoint * (1 + distance)  (wehere distance is represented as %)

    * bids/asks_limit/market/cancel_quantity_x = quantity (= bids/asks_limit/market/cancel_notional_x / bids/ask_price_x) of orders at bid/ask level_x
    
    * bids/asks_limit/market/cancel_cum_quantity_x = Cululative sum of quantities - i.e. bids/asks_limit/market/cancel_quantity_x
                                                     Example. cum_quantity_0 = quantity_0
                                                              cum_quantity_1 = quantity_0 + quantity_1
                                                              cum_quantity_2 = quantity_0 + quantity_1 + quantity_2
                                                              ...

    * bids/asks_limit/market/cancel_cum_notional_x = Cululative sum of notionals - i.e. bids/asks_limit/market/cancel_notional_x
                                                     Example. cum_notional_0 = notional_0
                                                              cum_notional_1 = notional_0 + notional_1
                                                              cum_notional_2 = notional_0 + notional_1 + notional_2
                                                              ...
    * bid_ask_imbalance_limit/market/cancel_notiona_x = Bid Ask Imbalance = (bid notional / (bid notional + ask notional))
                                                                          =  bids_limit/market/cancel_notional_x / (bids_limit/market/cancel_notional_x + asks_limit/market/cancel_notional_x)                      

#### DEFINE CONTROL PARAMETERS 

In [2]:
# Select which features/columns to include in ML Model feature set
# Select which "derived" features to include in ML Model feature set

# Select crypto assets interested in evaluating
INCLUDE_BTEC = True
INCLUDE_ETH = True
INCLUDE_ADA = True

# Select the orderbook timeframe's interested in
INCLUDE_5MIN_DATA = True
INCLUDE_1MIN_DATA = True
INCLUDE_1SEC_DATA = False

# Optional removing of feature/columns from provided crypto order book
# Remove cancel features/columns
REMOVE_CANCEL_FEATURES = True
# Remove market features/columns
REMOVE_MARKET_FEATURES = True
# Remove price distance feature/column
REMOVE_PRICE_DISTANCES = False
# Remove notional feature/column 
REMOVE_FINAL_NOTIONAL = True

# Optional NEW derived feature/columns that can be added to dataframe order book
# Option to add aboslute price levels to feature set (default dataframe shows delta price with mid price = distance)
ADD_ABSOLUTE_PRICE_LEVELS = True
# Option to add quantity at each level
ADD_QUANTITY_FEATURE = True
# Option to add CUMULATIVE quantity "across" levels
ADD_CUM_QUANTITY_FEATURE = True
# Option to add CUMULATIVE quantity "across" levels
ADD_CUM_NOTIONAL_FEATURE = True
# Option to add bid and ask IMBALANCE
ADD_BID_ASK_IMBALANCE = True
# ToDo: ADD MORE FEATURES (e.g. logarithmic returns)

# Level selection
# Select number of orderbook levels to include in feature set (max = 15)
MAX_ORDER_BOOK_LEVELS = 15   # Do not modify
NUM_ORDER_BOOK_LEVELS = 15

In [3]:
TURN_ON_PRINT_DEBUG = True
WRITE_TO_CSC_FILE = True

In [4]:
# Setting these options will allow for reviewing more of the DataFrames
pd.set_option('display.max_columns', 200)

#### Import the Crytpo Order Book Data

In [5]:
# dictionary to store crypto dataframes {BTEC, ETH, ADA}, {5min, 1min, 1sec}
crypto_ob_df_dict = {} 

# Based on the selection critero, read the csv crypto order book csv files into the dictrionary
if (INCLUDE_5MIN_DATA):
    if (INCLUDE_BTEC):
        crypto_ob_df_dict["BTC_5min"] = pd.read_csv(
            Path("./data/BTC_5min.csv"), 
            index_col='system_time', 
            infer_datetime_format=True, 
            parse_dates=True
            )
    if (INCLUDE_ETH):
        crypto_ob_df_dict["ETH_5min"] = pd.read_csv(
            Path("./data/ETH_5min.csv"), 
            index_col='system_time', 
            infer_datetime_format=True, 
            parse_dates=True
        )
    if (INCLUDE_ADA):
        crypto_ob_df_dict["ADA_5min"] = pd.read_csv(
            Path("./data/ADA_5min.csv"), 
            index_col='system_time', 
            infer_datetime_format=True, 
            parse_dates=True
        )
    
if (INCLUDE_1MIN_DATA):
    if (INCLUDE_BTEC):
        crypto_ob_df_dict["BTC_1min"] = pd.read_csv(
            Path("./data/BTC_1min.csv"), 
            index_col='system_time', 
            infer_datetime_format=True, 
            parse_dates=True
            )
    if (INCLUDE_ETH):
        crypto_ob_df_dict["ETH_1min"] = pd.read_csv(
            Path("./data/ETH_1min.csv"), 
            index_col='system_time', 
            infer_datetime_format=True, 
            parse_dates=True
        )
    if (INCLUDE_ADA):
        crypto_ob_df_dict["ADA_1min"] = pd.read_csv(
            Path("./data/ADA_1min.csv"), 
            index_col='system_time', 
            infer_datetime_format=True, 
            parse_dates=True
        )
    
if (INCLUDE_1SEC_DATA):
    if (INCLUDE_BTEC):
        crypto_ob_df_dict["BTC_1sec"] = pd.read_csv(
            Path("./data/BTC_1sec.csv"), 
            index_col='system_time', 
            infer_datetime_format=True, 
            parse_dates=True
            )
    if (INCLUDE_ETH):
        crypto_ob_df_dict["ETH_1sec"] = pd.read_csv(
            Path("./data/ETH_1sec.csv"), 
            index_col='system_time', 
            infer_datetime_format=True, 
            parse_dates=True
        )
    if (INCLUDE_ADA):
        crypto_ob_df_dict["ADA_1sec"] = pd.read_csv(
            Path("./data/ADA_1sec.csv"), 
            index_col='system_time', 
            infer_datetime_format=True, 
            parse_dates=True
        )    

In [6]:
# Debug data checkpoint

if (TURN_ON_PRINT_DEBUG):
    display(crypto_ob_df_dict.keys())
    
    for crypto_ob_key in crypto_ob_df_dict: 
        print(f"\nDebug {crypto_ob_key} dataframe after downloading/reading crypto provided csv files:")
        display(crypto_ob_df_dict[crypto_ob_key].shape)
        display(crypto_ob_df_dict[crypto_ob_key].head(n=5))

dict_keys(['BTC_5min', 'ETH_5min', 'ADA_5min', 'BTC_1min', 'ETH_1min', 'ADA_1min'])


Debug BTC_5min dataframe after downloading/reading crypto provided csv files:


(3367, 155)

,Unnamed: 0,midpoint,spread,buys,sells,bids_distance_0,bids_distance_1,bids_distance_2,bids_distance_3,bids_distance_4,bids_distance_5,bids_distance_6,bids_distance_7,bids_distance_8,bids_distance_9,bids_distance_10,bids_distance_11,bids_distance_12,bids_distance_13,bids_distance_14,bids_notional_0,bids_notional_1,bids_notional_2,bids_notional_3,bids_notional_4,bids_notional_5,bids_notional_6,bids_notional_7,bids_notional_8,bids_notional_9,bids_notional_10,bids_notional_11,bids_notional_12,bids_notional_13,bids_notional_14,bids_cancel_notional_0,bids_cancel_notional_1,bids_cancel_notional_2,bids_cancel_notional_3,bids_cancel_notional_4,bids_cancel_notional_5,bids_cancel_notional_6,bids_cancel_notional_7,bids_cancel_notional_8,bids_cancel_notional_9,bids_cancel_notional_10,bids_cancel_notional_11,bids_cancel_notional_12,bids_cancel_notional_13,bids_cancel_notional_14,bids_limit_notional_0,bids_limit_notional_1,bids_limit_notional_2,bids_limit_notional_3,bids_limit_notional_4,bids_limit_notional_5,bids_limit_notional_6,bids_limit_notional_7,bids_limit_notional_8,bids_limit_notional_9,bids_limit_notional_10,bids_limit_notional_11,bids_limit_notional_12,bids_limit_notional_13,bids_limit_notional_14,bids_market_notional_0,bids_market_notional_1,bids_market_notional_2,bids_market_notional_3,bids_market_notional_4,bids_market_notional_5,bids_market_notional_6,bids_market_notional_7,bids_market_notional_8,bids_market_notional_9,bids_market_notional_10,bids_market_notional_11,bids_market_notional_12,bids_market_notional_13,bids_market_notional_14,asks_distance_0,asks_distance_1,asks_distance_2,asks_distance_3,asks_distance_4,asks_distance_5,asks_distance_6,asks_distance_7,asks_distance_8,asks_distance_9,asks_distance_10,asks_distance_11,asks_distance_12,asks_distance_13,asks_distance_14,asks_notional_0,asks_notional_1,asks_notional_2,asks_notional_3,asks_notional_4,asks_notional_5,asks_notional_6,asks_notional_7,asks_notional_8,asks_notional_9,asks_notional_10,asks_notional_11,asks_notional_12,asks_notional_13,asks_notional_14,asks_cancel_notional_0,asks_cancel_notional_1,asks_cancel_notional_2,asks_cancel_notional_3,asks_cancel_notional_4,asks_cancel_notional_5,asks_cancel_notional_6,asks_cancel_notional_7,asks_cancel_notional_8,asks_cancel_notional_9,asks_cancel_notional_10,asks_cancel_notional_11,asks_cancel_notional_12,asks_cancel_notional_13,asks_cancel_notional_14,asks_limit_notional_0,asks_limit_notional_1,asks_limit_notional_2,asks_limit_notional_3,asks_limit_notional_4,asks_limit_notional_5,asks_limit_notional_6,asks_limit_notional_7,asks_limit_notional_8,asks_limit_notional_9,asks_limit_notional_10,asks_limit_notional_11,asks_limit_notional_12,asks_limit_notional_13,asks_limit_notional_14,asks_market_notional_0,asks_market_notional_1,asks_market_notional_2,asks_market_notional_3,asks_market_notional_4,asks_market_notional_5,asks_market_notional_6,asks_market_notional_7,asks_market_notional_8,asks_market_notional_9,asks_market_notional_10,asks_market_notional_11,asks_market_notional_12,asks_market_notional_13,asks_market_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:41.122161+00:00,0,55899.995,0.01,1.199304e+07,1.348616e+07,-8.944544e-08,-1.294276e-04,-0.000130,-0.000143,-0.000201,-0.000201,-0.000240,-0.000271,-0.000387,-0.000399,-0.000424,-0.000588,-0.000588,-0.000629,-0.000727,18997.929688,746.049988,1117.859985,2794.600098,14100.320312,5588.879883,187.220001,27942.429688,13969.589844,3362.449951,27044.119141,2793.360107,25698.880859,2793.239990,500.000000,1.309910e+03,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,624.530029,0.00000,0.0,0.0,0.000000,0.000000,0.0,2.030784e+04,746.049988,1117.859985,2794.600098,14100.320312,5588.879883,187.220001,27942.429688,14594.120117,3362.449951,27044.119141,2793.360107,25698.880859,2793.239990,500.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,


Debug ETH_5min dataframe after downloading/reading crypto provided csv files:


...

(17110, 155)

,Unnamed: 0,midpoint,spread,buys,sells,bids_distance_0,bids_distance_1,bids_distance_2,bids_distance_3,bids_distance_4,bids_distance_5,bids_distance_6,bids_distance_7,bids_distance_8,bids_distance_9,bids_distance_10,bids_distance_11,bids_distance_12,bids_distance_13,bids_distance_14,bids_notional_0,bids_notional_1,bids_notional_2,bids_notional_3,bids_notional_4,bids_notional_5,bids_notional_6,bids_notional_7,bids_notional_8,bids_notional_9,bids_notional_10,bids_notional_11,bids_notional_12,bids_notional_13,bids_notional_14,bids_cancel_notional_0,bids_cancel_notional_1,bids_cancel_notional_2,bids_cancel_notional_3,bids_cancel_notional_4,bids_cancel_notional_5,bids_cancel_notional_6,bids_cancel_notional_7,bids_cancel_notional_8,bids_cancel_notional_9,bids_cancel_notional_10,bids_cancel_notional_11,bids_cancel_notional_12,bids_cancel_notional_13,bids_cancel_notional_14,bids_limit_notional_0,bids_limit_notional_1,bids_limit_notional_2,bids_limit_notional_3,bids_limit_notional_4,bids_limit_notional_5,bids_limit_notional_6,bids_limit_notional_7,bids_limit_notional_8,bids_limit_notional_9,bids_limit_notional_10,bids_limit_notional_11,bids_limit_notional_12,bids_limit_notional_13,bids_limit_notional_14,bids_market_notional_0,bids_market_notional_1,bids_market_notional_2,bids_market_notional_3,bids_market_notional_4,bids_market_notional_5,bids_market_notional_6,bids_market_notional_7,bids_market_notional_8,bids_market_notional_9,bids_market_notional_10,bids_market_notional_11,bids_market_notional_12,bids_market_notional_13,bids_market_notional_14,asks_distance_0,asks_distance_1,asks_distance_2,asks_distance_3,asks_distance_4,asks_distance_5,asks_distance_6,asks_distance_7,asks_distance_8,asks_distance_9,asks_distance_10,asks_distance_11,asks_distance_12,asks_distance_13,asks_distance_14,asks_notional_0,asks_notional_1,asks_notional_2,asks_notional_3,asks_notional_4,asks_notional_5,asks_notional_6,asks_notional_7,asks_notional_8,asks_notional_9,asks_notional_10,asks_notional_11,asks_notional_12,asks_notional_13,asks_notional_14,asks_cancel_notional_0,asks_cancel_notional_1,asks_cancel_notional_2,asks_cancel_notional_3,asks_cancel_notional_4,asks_cancel_notional_5,asks_cancel_notional_6,asks_cancel_notional_7,asks_cancel_notional_8,asks_cancel_notional_9,asks_cancel_notional_10,asks_cancel_notional_11,asks_cancel_notional_12,asks_cancel_notional_13,asks_cancel_notional_14,asks_limit_notional_0,asks_limit_notional_1,asks_limit_notional_2,asks_limit_notional_3,asks_limit_notional_4,asks_limit_notional_5,asks_limit_notional_6,asks_limit_notional_7,asks_limit_notional_8,asks_limit_notional_9,asks_limit_notional_10,asks_limit_notional_11,asks_limit_notional_12,asks_limit_notional_13,asks_limit_notional_14,asks_market_notional_0,asks_market_notional_1,asks_market_notional_2,asks_market_notional_3,asks_market_notional_4,asks_market_notional_5,asks_market_notional_6,asks_market_notional_7,asks_market_notional_8,asks_market_notional_9,asks_market_notional_10,asks_market_notional_11,asks_market_notional_12,asks_market_notional_13,asks_market_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:33:49.861733+00:00,0,1965.845,0.01,875154.482918,1.684774e+06,-0.000003,-0.000155,-0.000160,-0.000257,-0.000262,-0.000277,-0.000425,-0.000430,-0.000455,-0.000506,-0.000532,-0.000547,-0.000593,-0.000613,-0.000674,44445.929688,7766.830078,196.550003,9972.139648,42644.878906,34927.250000,19519.410156,12016.940430,514.400024,751.210022,2057.439941,760.760010,4954.089844,4995.359863,1998.489990,42265.558594,1883.47998,29482.949219,0.000000,41288.148438,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.000000,86711.492188,9650.309570,29679.500000,9972.139648,83933.031250,34927.250000,19519.410156,14049.750000,514.400024,751.210022,2057.439941,760.760010,4954.089844,4995.359863,1998.489990,0.0,0.0,0.0,0.0,0.0,0


Debug ADA_1min dataframe after downloading/reading crypto provided csv files:


(17109, 155)

,Unnamed: 0,midpoint,spread,buys,sells,bids_distance_0,bids_distance_1,bids_distance_2,bids_distance_3,bids_distance_4,bids_distance_5,bids_distance_6,bids_distance_7,bids_distance_8,bids_distance_9,bids_distance_10,bids_distance_11,bids_distance_12,bids_distance_13,bids_distance_14,bids_notional_0,bids_notional_1,bids_notional_2,bids_notional_3,bids_notional_4,bids_notional_5,bids_notional_6,bids_notional_7,bids_notional_8,bids_notional_9,bids_notional_10,bids_notional_11,bids_notional_12,bids_notional_13,bids_notional_14,bids_cancel_notional_0,bids_cancel_notional_1,bids_cancel_notional_2,bids_cancel_notional_3,bids_cancel_notional_4,bids_cancel_notional_5,bids_cancel_notional_6,bids_cancel_notional_7,bids_cancel_notional_8,bids_cancel_notional_9,bids_cancel_notional_10,bids_cancel_notional_11,bids_cancel_notional_12,bids_cancel_notional_13,bids_cancel_notional_14,bids_limit_notional_0,bids_limit_notional_1,bids_limit_notional_2,bids_limit_notional_3,bids_limit_notional_4,bids_limit_notional_5,bids_limit_notional_6,bids_limit_notional_7,bids_limit_notional_8,bids_limit_notional_9,bids_limit_notional_10,bids_limit_notional_11,bids_limit_notional_12,bids_limit_notional_13,bids_limit_notional_14,bids_market_notional_0,bids_market_notional_1,bids_market_notional_2,bids_market_notional_3,bids_market_notional_4,bids_market_notional_5,bids_market_notional_6,bids_market_notional_7,bids_market_notional_8,bids_market_notional_9,bids_market_notional_10,bids_market_notional_11,bids_market_notional_12,bids_market_notional_13,bids_market_notional_14,asks_distance_0,asks_distance_1,asks_distance_2,asks_distance_3,asks_distance_4,asks_distance_5,asks_distance_6,asks_distance_7,asks_distance_8,asks_distance_9,asks_distance_10,asks_distance_11,asks_distance_12,asks_distance_13,asks_distance_14,asks_notional_0,asks_notional_1,asks_notional_2,asks_notional_3,asks_notional_4,asks_notional_5,asks_notional_6,asks_notional_7,asks_notional_8,asks_notional_9,asks_notional_10,asks_notional_11,asks_notional_12,asks_notional_13,asks_notional_14,asks_cancel_notional_0,asks_cancel_notional_1,asks_cancel_notional_2,asks_cancel_notional_3,asks_cancel_notional_4,asks_cancel_notional_5,asks_cancel_notional_6,asks_cancel_notional_7,asks_cancel_notional_8,asks_cancel_notional_9,asks_cancel_notional_10,asks_cancel_notional_11,asks_cancel_notional_12,asks_cancel_notional_13,asks_cancel_notional_14,asks_limit_notional_0,asks_limit_notional_1,asks_limit_notional_2,asks_limit_notional_3,asks_limit_notional_4,asks_limit_notional_5,asks_limit_notional_6,asks_limit_notional_7,asks_limit_notional_8,asks_limit_notional_9,asks_limit_notional_10,asks_limit_notional_11,asks_limit_notional_12,asks_limit_notional_13,asks_limit_notional_14,asks_market_notional_0,asks_market_notional_1,asks_market_notional_2,asks_market_notional_3,asks_market_notional_4,asks_market_notional_5,asks_market_notional_6,asks_market_notional_7,asks_market_notional_8,asks_market_notional_9,asks_market_notional_10,asks_market_notional_11,asks_market_notional_12,asks_market_notional_13,asks_market_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:33:59.055697+00:00,0,1.16205,0.0001,56936.467913,258248.957367,-0.000043,-0.000129,-0.000301,-0.000473,-0.000731,-0.000818,-0.000904,-0.000990,-0.001162,-0.001248,-0.001334,-0.001420,-0.001506,-0.001592,-0.001678,876.869995,5984.169922,5.810000,18.240000,19844.640625,168.360001,5914.439941,1227.089966,5723.609863,2724.550049,7897.859863,25.799999,11387.959961,37.220001,26919.900391,242372.171875,23587.300781,28962.919922,19644.429688,17385.220703,210554.281250,70002.617188,44777.421875,8705.879883,126408.992188,16750.339844,126284.359375,42670.550781,119406.406250,112427.250000,244337.500000,29571.470703,28968.730469,19662.660156,37229.859375,210722.640625,75917.062500,46004.519531,14429.490234,129133.539062,24648.210938

#### PRE-PROCESS ORDER BOOK FEATURES/COLUMNS

In [7]:
# Drop the "Unnamed" column

for crypto_ob_key in crypto_ob_df_dict: 
    crypto_ob_df_dict[crypto_ob_key].drop("Unnamed: 0", axis=1, inplace=True)
    
    if (TURN_ON_PRINT_DEBUG):
        print(f"\nDebug {crypto_ob_key} dataframe after dropping Unnamed column:")
        display(crypto_ob_df_dict[crypto_ob_key].shape)
        display(crypto_ob_df_dict[crypto_ob_key].head(n=1))


Debug BTC_5min dataframe after dropping Unnamed column:


(3367, 154)

,midpoint,spread,buys,sells,bids_distance_0,bids_distance_1,bids_distance_2,bids_distance_3,bids_distance_4,bids_distance_5,bids_distance_6,bids_distance_7,bids_distance_8,bids_distance_9,bids_distance_10,bids_distance_11,bids_distance_12,bids_distance_13,bids_distance_14,bids_notional_0,bids_notional_1,bids_notional_2,bids_notional_3,bids_notional_4,bids_notional_5,bids_notional_6,bids_notional_7,bids_notional_8,bids_notional_9,bids_notional_10,bids_notional_11,bids_notional_12,bids_notional_13,bids_notional_14,bids_cancel_notional_0,bids_cancel_notional_1,bids_cancel_notional_2,bids_cancel_notional_3,bids_cancel_notional_4,bids_cancel_notional_5,bids_cancel_notional_6,bids_cancel_notional_7,bids_cancel_notional_8,bids_cancel_notional_9,bids_cancel_notional_10,bids_cancel_notional_11,bids_cancel_notional_12,bids_cancel_notional_13,bids_cancel_notional_14,bids_limit_notional_0,bids_limit_notional_1,bids_limit_notional_2,bids_limit_notional_3,bids_limit_notional_4,bids_limit_notional_5,bids_limit_notional_6,bids_limit_notional_7,bids_limit_notional_8,bids_limit_notional_9,bids_limit_notional_10,bids_limit_notional_11,bids_limit_notional_12,bids_limit_notional_13,bids_limit_notional_14,bids_market_notional_0,bids_market_notional_1,bids_market_notional_2,bids_market_notional_3,bids_market_notional_4,bids_market_notional_5,bids_market_notional_6,bids_market_notional_7,bids_market_notional_8,bids_market_notional_9,bids_market_notional_10,bids_market_notional_11,bids_market_notional_12,bids_market_notional_13,bids_market_notional_14,asks_distance_0,asks_distance_1,asks_distance_2,asks_distance_3,asks_distance_4,asks_distance_5,asks_distance_6,asks_distance_7,asks_distance_8,asks_distance_9,asks_distance_10,asks_distance_11,asks_distance_12,asks_distance_13,asks_distance_14,asks_notional_0,asks_notional_1,asks_notional_2,asks_notional_3,asks_notional_4,asks_notional_5,asks_notional_6,asks_notional_7,asks_notional_8,asks_notional_9,asks_notional_10,asks_notional_11,asks_notional_12,asks_notional_13,asks_notional_14,asks_cancel_notional_0,asks_cancel_notional_1,asks_cancel_notional_2,asks_cancel_notional_3,asks_cancel_notional_4,asks_cancel_notional_5,asks_cancel_notional_6,asks_cancel_notional_7,asks_cancel_notional_8,asks_cancel_notional_9,asks_cancel_notional_10,asks_cancel_notional_11,asks_cancel_notional_12,asks_cancel_notional_13,asks_cancel_notional_14,asks_limit_notional_0,asks_limit_notional_1,asks_limit_notional_2,asks_limit_notional_3,asks_limit_notional_4,asks_limit_notional_5,asks_limit_notional_6,asks_limit_notional_7,asks_limit_notional_8,asks_limit_notional_9,asks_limit_notional_10,asks_limit_notional_11,asks_limit_notional_12,asks_limit_notional_13,asks_limit_notional_14,asks_market_notional_0,asks_market_notional_1,asks_market_notional_2,asks_market_notional_3,asks_market_notional_4,asks_market_notional_5,asks_market_notional_6,asks_market_notional_7,asks_market_notional_8,asks_market_notional_9,asks_market_notional_10,asks_market_notional_11,asks_market_notional_12,asks_market_notional_13,asks_market_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:41.122161+00:00,55899.995,0.01,1.199304e+07,1.348616e+07,-8.944544e-08,-0.000129,-0.00013,-0.000143,-0.000201,-0.000201,-0.00024,-0.000271,-0.000387,-0.000399,-0.000424,-0.000588,-0.000588,-0.000629,-0.000727,18997.929688,746.049988,1117.859985,2794.600098,14100.320312,5588.879883,187.220001,27942.429688,13969.589844,3362.449951,27044.119141,2793.360107,25698.880859,2793.23999,500.0,1309.910034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,624.530029,0.0,0.0,0.0,0.0,0.0,0.0,20307.839844,746.049988,1117.859985,2794.600098,14100.320312,5588.879883,187.220001,27942.429688,14594.120117,3362.449951,27044.119141,2793.360107,25698.880859,2793.23999,500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.944544e-08,0.000033,0.000278,0.000345,0


Debug ETH_5min dataframe after dropping Unnamed column:


(3365, 154)

...

(17110, 154)

,midpoint,spread,buys,sells,bids_distance_0,bids_distance_1,bids_distance_2,bids_distance_3,bids_distance_4,bids_distance_5,bids_distance_6,bids_distance_7,bids_distance_8,bids_distance_9,bids_distance_10,bids_distance_11,bids_distance_12,bids_distance_13,bids_distance_14,bids_notional_0,bids_notional_1,bids_notional_2,bids_notional_3,bids_notional_4,bids_notional_5,bids_notional_6,bids_notional_7,bids_notional_8,bids_notional_9,bids_notional_10,bids_notional_11,bids_notional_12,bids_notional_13,bids_notional_14,bids_cancel_notional_0,bids_cancel_notional_1,bids_cancel_notional_2,bids_cancel_notional_3,bids_cancel_notional_4,bids_cancel_notional_5,bids_cancel_notional_6,bids_cancel_notional_7,bids_cancel_notional_8,bids_cancel_notional_9,bids_cancel_notional_10,bids_cancel_notional_11,bids_cancel_notional_12,bids_cancel_notional_13,bids_cancel_notional_14,bids_limit_notional_0,bids_limit_notional_1,bids_limit_notional_2,bids_limit_notional_3,bids_limit_notional_4,bids_limit_notional_5,bids_limit_notional_6,bids_limit_notional_7,bids_limit_notional_8,bids_limit_notional_9,bids_limit_notional_10,bids_limit_notional_11,bids_limit_notional_12,bids_limit_notional_13,bids_limit_notional_14,bids_market_notional_0,bids_market_notional_1,bids_market_notional_2,bids_market_notional_3,bids_market_notional_4,bids_market_notional_5,bids_market_notional_6,bids_market_notional_7,bids_market_notional_8,bids_market_notional_9,bids_market_notional_10,bids_market_notional_11,bids_market_notional_12,bids_market_notional_13,bids_market_notional_14,asks_distance_0,asks_distance_1,asks_distance_2,asks_distance_3,asks_distance_4,asks_distance_5,asks_distance_6,asks_distance_7,asks_distance_8,asks_distance_9,asks_distance_10,asks_distance_11,asks_distance_12,asks_distance_13,asks_distance_14,asks_notional_0,asks_notional_1,asks_notional_2,asks_notional_3,asks_notional_4,asks_notional_5,asks_notional_6,asks_notional_7,asks_notional_8,asks_notional_9,asks_notional_10,asks_notional_11,asks_notional_12,asks_notional_13,asks_notional_14,asks_cancel_notional_0,asks_cancel_notional_1,asks_cancel_notional_2,asks_cancel_notional_3,asks_cancel_notional_4,asks_cancel_notional_5,asks_cancel_notional_6,asks_cancel_notional_7,asks_cancel_notional_8,asks_cancel_notional_9,asks_cancel_notional_10,asks_cancel_notional_11,asks_cancel_notional_12,asks_cancel_notional_13,asks_cancel_notional_14,asks_limit_notional_0,asks_limit_notional_1,asks_limit_notional_2,asks_limit_notional_3,asks_limit_notional_4,asks_limit_notional_5,asks_limit_notional_6,asks_limit_notional_7,asks_limit_notional_8,asks_limit_notional_9,asks_limit_notional_10,asks_limit_notional_11,asks_limit_notional_12,asks_limit_notional_13,asks_limit_notional_14,asks_market_notional_0,asks_market_notional_1,asks_market_notional_2,asks_market_notional_3,asks_market_notional_4,asks_market_notional_5,asks_market_notional_6,asks_market_notional_7,asks_market_notional_8,asks_market_notional_9,asks_market_notional_10,asks_market_notional_11,asks_market_notional_12,asks_market_notional_13,asks_market_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:33:49.861733+00:00,1965.845,0.01,875154.482918,1.684774e+06,-0.000003,-0.000155,-0.00016,-0.000257,-0.000262,-0.000277,-0.000425,-0.00043,-0.000455,-0.000506,-0.000532,-0.000547,-0.000593,-0.000613,-0.000674,44445.929688,7766.830078,196.550003,9972.139648,42644.878906,34927.25,19519.410156,12016.94043,514.400024,751.210022,2057.439941,760.76001,4954.089844,4995.359863,1998.48999,42265.558594,1883.47998,29482.949219,0.0,41288.148438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,86711.492188,9650.30957,29679.5,9972.139648,83933.03125,34927.25,19519.410156,14049.75,514.400024,751.210022,2057.439941,760.76001,4954.089844,4995.359863,1998.48999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2032.810059,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000003,0.000135,0.00014,0.0001


Debug ADA_1min dataframe after dropping Unnamed column:


(17109, 154)

,midpoint,spread,buys,sells,bids_distance_0,bids_distance_1,bids_distance_2,bids_distance_3,bids_distance_4,bids_distance_5,bids_distance_6,bids_distance_7,bids_distance_8,bids_distance_9,bids_distance_10,bids_distance_11,bids_distance_12,bids_distance_13,bids_distance_14,bids_notional_0,bids_notional_1,bids_notional_2,bids_notional_3,bids_notional_4,bids_notional_5,bids_notional_6,bids_notional_7,bids_notional_8,bids_notional_9,bids_notional_10,bids_notional_11,bids_notional_12,bids_notional_13,bids_notional_14,bids_cancel_notional_0,bids_cancel_notional_1,bids_cancel_notional_2,bids_cancel_notional_3,bids_cancel_notional_4,bids_cancel_notional_5,bids_cancel_notional_6,bids_cancel_notional_7,bids_cancel_notional_8,bids_cancel_notional_9,bids_cancel_notional_10,bids_cancel_notional_11,bids_cancel_notional_12,bids_cancel_notional_13,bids_cancel_notional_14,bids_limit_notional_0,bids_limit_notional_1,bids_limit_notional_2,bids_limit_notional_3,bids_limit_notional_4,bids_limit_notional_5,bids_limit_notional_6,bids_limit_notional_7,bids_limit_notional_8,bids_limit_notional_9,bids_limit_notional_10,bids_limit_notional_11,bids_limit_notional_12,bids_limit_notional_13,bids_limit_notional_14,bids_market_notional_0,bids_market_notional_1,bids_market_notional_2,bids_market_notional_3,bids_market_notional_4,bids_market_notional_5,bids_market_notional_6,bids_market_notional_7,bids_market_notional_8,bids_market_notional_9,bids_market_notional_10,bids_market_notional_11,bids_market_notional_12,bids_market_notional_13,bids_market_notional_14,asks_distance_0,asks_distance_1,asks_distance_2,asks_distance_3,asks_distance_4,asks_distance_5,asks_distance_6,asks_distance_7,asks_distance_8,asks_distance_9,asks_distance_10,asks_distance_11,asks_distance_12,asks_distance_13,asks_distance_14,asks_notional_0,asks_notional_1,asks_notional_2,asks_notional_3,asks_notional_4,asks_notional_5,asks_notional_6,asks_notional_7,asks_notional_8,asks_notional_9,asks_notional_10,asks_notional_11,asks_notional_12,asks_notional_13,asks_notional_14,asks_cancel_notional_0,asks_cancel_notional_1,asks_cancel_notional_2,asks_cancel_notional_3,asks_cancel_notional_4,asks_cancel_notional_5,asks_cancel_notional_6,asks_cancel_notional_7,asks_cancel_notional_8,asks_cancel_notional_9,asks_cancel_notional_10,asks_cancel_notional_11,asks_cancel_notional_12,asks_cancel_notional_13,asks_cancel_notional_14,asks_limit_notional_0,asks_limit_notional_1,asks_limit_notional_2,asks_limit_notional_3,asks_limit_notional_4,asks_limit_notional_5,asks_limit_notional_6,asks_limit_notional_7,asks_limit_notional_8,asks_limit_notional_9,asks_limit_notional_10,asks_limit_notional_11,asks_limit_notional_12,asks_limit_notional_13,asks_limit_notional_14,asks_market_notional_0,asks_market_notional_1,asks_market_notional_2,asks_market_notional_3,asks_market_notional_4,asks_market_notional_5,asks_market_notional_6,asks_market_notional_7,asks_market_notional_8,asks_market_notional_9,asks_market_notional_10,asks_market_notional_11,asks_market_notional_12,asks_market_notional_13,asks_market_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:33:59.055697+00:00,1.16205,0.0001,56936.467913,258248.957367,-0.000043,-0.000129,-0.000301,-0.000473,-0.000731,-0.000818,-0.000904,-0.00099,-0.001162,-0.001248,-0.001334,-0.00142,-0.001506,-0.001592,-0.001678,876.869995,5984.169922,5.81,18.24,19844.640625,168.360001,5914.439941,1227.089966,5723.609863,2724.550049,7897.859863,25.799999,11387.959961,37.220001,26919.900391,242372.171875,23587.300781,28962.919922,19644.429688,17385.220703,210554.28125,70002.617188,44777.421875,8705.879883,126408.992188,16750.339844,126284.359375,42670.550781,119406.40625,112427.25,244337.5,29571.470703,28968.730469,19662.660156,37229.859375,210722.640625,75917.0625,46004.519531,14429.490234,129133.539062,24648.210938,126310.15625,54058.519531,119443.632

In [8]:
# Rearrange columns in crypto dataframes
# The default arrangement of the order book data provided list all the levels for each feature in order of level
#     (e.g. bids_distance_0, bids_dstance_1, ..., bids_distance_14, bids_notional_0, bids_notional_1, ..., bids_notional_14, ... )
# Prefer to have all the same level features listed together in the columns (makes viewing of the dataframes easier)
#     (e.g. bids_distance_0, bids_notional_0, bids_price_0, asks_distance_0, ... bids_distance_1, ...)

# Keep the first 4 columns in the same position
new_coloumns_lst = ["midpoint", "spread", "buys", "sells"]

start_col_index = 4 # incrementing variable to index/location of where to move the columns to
bid_ask_str_lst = ["bids", "asks"] 
level_str_lst = ["distance", "limit_notional", "market_notional", "cancel_notional", "notional"]
# (Note: notional = limit_notional - market_notional - cancel_notional)

# Add the column strings to the list in the level order (e.g. bids_distance_0, bids_notional_0, bids_price_0, asks_distance_0, ... bids_distance_1, ...)
for level_num in range(MAX_ORDER_BOOK_LEVELS):
    for bid_ask_str in bid_ask_str_lst:
        for level_str in level_str_lst:
            column_name_str_to_move = bid_ask_str + "_" + level_str + "_" + str(level_num)
            new_coloumns_lst.append(column_name_str_to_move)

# Reorder the columns for each of the cryptp dataframes
for crypto_ob_key in crypto_ob_df_dict:
    crypto_ob_df_dict[crypto_ob_key] = crypto_ob_df_dict[crypto_ob_key].reindex(columns=new_coloumns_lst)

    if (TURN_ON_PRINT_DEBUG):
        print(f"\nDebug {crypto_ob_key} dataframe after rearranging columns:")
        display(crypto_ob_df_dict[crypto_ob_key].shape)
        display(crypto_ob_df_dict[crypto_ob_key].head(n=1))                


Debug BTC_5min dataframe after rearranging columns:


(3367, 154)

,midpoint,spread,buys,sells,bids_distance_0,bids_limit_notional_0,bids_market_notional_0,bids_cancel_notional_0,bids_notional_0,asks_distance_0,asks_limit_notional_0,asks_market_notional_0,asks_cancel_notional_0,asks_notional_0,bids_distance_1,bids_limit_notional_1,bids_market_notional_1,bids_cancel_notional_1,bids_notional_1,asks_distance_1,asks_limit_notional_1,asks_market_notional_1,asks_cancel_notional_1,asks_notional_1,bids_distance_2,bids_limit_notional_2,bids_market_notional_2,bids_cancel_notional_2,bids_notional_2,asks_distance_2,asks_limit_notional_2,asks_market_notional_2,asks_cancel_notional_2,asks_notional_2,bids_distance_3,bids_limit_notional_3,bids_market_notional_3,bids_cancel_notional_3,bids_notional_3,asks_distance_3,asks_limit_notional_3,asks_market_notional_3,asks_cancel_notional_3,asks_notional_3,bids_distance_4,bids_limit_notional_4,bids_market_notional_4,bids_cancel_notional_4,bids_notional_4,asks_distance_4,asks_limit_notional_4,asks_market_notional_4,asks_cancel_notional_4,asks_notional_4,bids_distance_5,bids_limit_notional_5,bids_market_notional_5,bids_cancel_notional_5,bids_notional_5,asks_distance_5,asks_limit_notional_5,asks_market_notional_5,asks_cancel_notional_5,asks_notional_5,bids_distance_6,bids_limit_notional_6,bids_market_notional_6,bids_cancel_notional_6,bids_notional_6,asks_distance_6,asks_limit_notional_6,asks_market_notional_6,asks_cancel_notional_6,asks_notional_6,bids_distance_7,bids_limit_notional_7,bids_market_notional_7,bids_cancel_notional_7,bids_notional_7,asks_distance_7,asks_limit_notional_7,asks_market_notional_7,asks_cancel_notional_7,asks_notional_7,bids_distance_8,bids_limit_notional_8,bids_market_notional_8,bids_cancel_notional_8,bids_notional_8,asks_distance_8,asks_limit_notional_8,asks_market_notional_8,asks_cancel_notional_8,asks_notional_8,bids_distance_9,bids_limit_notional_9,bids_market_notional_9,bids_cancel_notional_9,bids_notional_9,asks_distance_9,asks_limit_notional_9,asks_market_notional_9,asks_cancel_notional_9,asks_notional_9,bids_distance_10,bids_limit_notional_10,bids_market_notional_10,bids_cancel_notional_10,bids_notional_10,asks_distance_10,asks_limit_notional_10,asks_market_notional_10,asks_cancel_notional_10,asks_notional_10,bids_distance_11,bids_limit_notional_11,bids_market_notional_11,bids_cancel_notional_11,bids_notional_11,asks_distance_11,asks_limit_notional_11,asks_market_notional_11,asks_cancel_notional_11,asks_notional_11,bids_distance_12,bids_limit_notional_12,bids_market_notional_12,bids_cancel_notional_12,bids_notional_12,asks_distance_12,asks_limit_notional_12,asks_market_notional_12,asks_cancel_notional_12,asks_notional_12,bids_distance_13,bids_limit_notional_13,bids_market_notional_13,bids_cancel_notional_13,bids_notional_13,asks_distance_13,asks_limit_notional_13,asks_market_notional_13,asks_cancel_notional_13,asks_notional_13,bids_distance_14,bids_limit_notional_14,bids_market_notional_14,bids_cancel_notional_14,bids_notional_14,asks_distance_14,asks_limit_notional_14,asks_market_notional_14,asks_cancel_notional_14,asks_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:41.122161+00:00,55899.995,0.01,1.199304e+07,1.348616e+07,-8.944544e-08,20307.839844,0.0,1309.910034,18997.929688,8.944544e-08,2208291.0,1986.449951,1107645.375,1098659.25,-0.000129,746.049988,0.0,0.0,746.049988,0.000033,345.220001,0.0,0.0,345.220001,-0.00013,1117.859985,0.0,0.0,1117.859985,0.000278,107.75,0.0,0.0,107.75,-0.000143,2794.600098,0.0,0.0,2794.600098,0.000345,27026.910156,0.0,26445.349609,581.559998,-0.000201,14100.320312,0.0,0.0,14100.320312,0.000364,283.829987,0.0,0.0,283.829987,-0.000201,5588.879883,0.0,0.0,5588.879883,0.000421,3511.439941,312.040009,715.26001,2484.139893,-0.00024,187.220001,0.0,0.0,187.220001,0.0005,2164.22998,0.0,0.0,2164.22998,-0.000271,27942.429688,0.0,0.0,27942.429688,0.000573,19576.220703,0.0,


Debug ETH_5min dataframe after rearranging columns:


(3365, 154)

...

(17110, 154)

,midpoint,spread,buys,sells,bids_distance_0,bids_limit_notional_0,bids_market_notional_0,bids_cancel_notional_0,bids_notional_0,asks_distance_0,asks_limit_notional_0,asks_market_notional_0,asks_cancel_notional_0,asks_notional_0,bids_distance_1,bids_limit_notional_1,bids_market_notional_1,bids_cancel_notional_1,bids_notional_1,asks_distance_1,asks_limit_notional_1,asks_market_notional_1,asks_cancel_notional_1,asks_notional_1,bids_distance_2,bids_limit_notional_2,bids_market_notional_2,bids_cancel_notional_2,bids_notional_2,asks_distance_2,asks_limit_notional_2,asks_market_notional_2,asks_cancel_notional_2,asks_notional_2,bids_distance_3,bids_limit_notional_3,bids_market_notional_3,bids_cancel_notional_3,bids_notional_3,asks_distance_3,asks_limit_notional_3,asks_market_notional_3,asks_cancel_notional_3,asks_notional_3,bids_distance_4,bids_limit_notional_4,bids_market_notional_4,bids_cancel_notional_4,bids_notional_4,asks_distance_4,asks_limit_notional_4,asks_market_notional_4,asks_cancel_notional_4,asks_notional_4,bids_distance_5,bids_limit_notional_5,bids_market_notional_5,bids_cancel_notional_5,bids_notional_5,asks_distance_5,asks_limit_notional_5,asks_market_notional_5,asks_cancel_notional_5,asks_notional_5,bids_distance_6,bids_limit_notional_6,bids_market_notional_6,bids_cancel_notional_6,bids_notional_6,asks_distance_6,asks_limit_notional_6,asks_market_notional_6,asks_cancel_notional_6,asks_notional_6,bids_distance_7,bids_limit_notional_7,bids_market_notional_7,bids_cancel_notional_7,bids_notional_7,asks_distance_7,asks_limit_notional_7,asks_market_notional_7,asks_cancel_notional_7,asks_notional_7,bids_distance_8,bids_limit_notional_8,bids_market_notional_8,bids_cancel_notional_8,bids_notional_8,asks_distance_8,asks_limit_notional_8,asks_market_notional_8,asks_cancel_notional_8,asks_notional_8,bids_distance_9,bids_limit_notional_9,bids_market_notional_9,bids_cancel_notional_9,bids_notional_9,asks_distance_9,asks_limit_notional_9,asks_market_notional_9,asks_cancel_notional_9,asks_notional_9,bids_distance_10,bids_limit_notional_10,bids_market_notional_10,bids_cancel_notional_10,bids_notional_10,asks_distance_10,asks_limit_notional_10,asks_market_notional_10,asks_cancel_notional_10,asks_notional_10,bids_distance_11,bids_limit_notional_11,bids_market_notional_11,bids_cancel_notional_11,bids_notional_11,asks_distance_11,asks_limit_notional_11,asks_market_notional_11,asks_cancel_notional_11,asks_notional_11,bids_distance_12,bids_limit_notional_12,bids_market_notional_12,bids_cancel_notional_12,bids_notional_12,asks_distance_12,asks_limit_notional_12,asks_market_notional_12,asks_cancel_notional_12,asks_notional_12,bids_distance_13,bids_limit_notional_13,bids_market_notional_13,bids_cancel_notional_13,bids_notional_13,asks_distance_13,asks_limit_notional_13,asks_market_notional_13,asks_cancel_notional_13,asks_notional_13,bids_distance_14,bids_limit_notional_14,bids_market_notional_14,bids_cancel_notional_14,bids_notional_14,asks_distance_14,asks_limit_notional_14,asks_market_notional_14,asks_cancel_notional_14,asks_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:33:49.861733+00:00,1965.845,0.01,875154.482918,1.684774e+06,-0.000003,86711.492188,0.0,42265.558594,44445.929688,0.000003,20005.570312,20004.490234,0.0,1.08,-0.000155,9650.30957,0.0,1883.47998,7766.830078,0.000135,3234.25,0.0,0.0,3234.25,-0.00016,29679.5,0.0,29482.949219,196.550003,0.00014,20005.310547,0.0,0.0,20005.310547,-0.000257,9972.139648,0.0,0.0,9972.139648,0.000196,4350.029785,0.0,0.0,4350.029785,-0.000262,83933.03125,0.0,41288.148438,42644.878906,0.000242,5839.970215,0.0,1966.319946,3873.649902,-0.000277,34927.25,0.0,0.0,34927.25,0.000277,20005.439453,0.0,0.0,20005.439453,-0.000425,19519.410156,0.0,0.0,19519.410156,0.000415,3874.320068,0.0,0.0,3874.320068,-0.00043,14049.75,2032.810059,0.0,12016.94043,0.000455,731.830017,0.0


Debug ADA_1min dataframe after rearranging columns:


(17109, 154)

,midpoint,spread,buys,sells,bids_distance_0,bids_limit_notional_0,bids_market_notional_0,bids_cancel_notional_0,bids_notional_0,asks_distance_0,asks_limit_notional_0,asks_market_notional_0,asks_cancel_notional_0,asks_notional_0,bids_distance_1,bids_limit_notional_1,bids_market_notional_1,bids_cancel_notional_1,bids_notional_1,asks_distance_1,asks_limit_notional_1,asks_market_notional_1,asks_cancel_notional_1,asks_notional_1,bids_distance_2,bids_limit_notional_2,bids_market_notional_2,bids_cancel_notional_2,bids_notional_2,asks_distance_2,asks_limit_notional_2,asks_market_notional_2,asks_cancel_notional_2,asks_notional_2,bids_distance_3,bids_limit_notional_3,bids_market_notional_3,bids_cancel_notional_3,bids_notional_3,asks_distance_3,asks_limit_notional_3,asks_market_notional_3,asks_cancel_notional_3,asks_notional_3,bids_distance_4,bids_limit_notional_4,bids_market_notional_4,bids_cancel_notional_4,bids_notional_4,asks_distance_4,asks_limit_notional_4,asks_market_notional_4,asks_cancel_notional_4,asks_notional_4,bids_distance_5,bids_limit_notional_5,bids_market_notional_5,bids_cancel_notional_5,bids_notional_5,asks_distance_5,asks_limit_notional_5,asks_market_notional_5,asks_cancel_notional_5,asks_notional_5,bids_distance_6,bids_limit_notional_6,bids_market_notional_6,bids_cancel_notional_6,bids_notional_6,asks_distance_6,asks_limit_notional_6,asks_market_notional_6,asks_cancel_notional_6,asks_notional_6,bids_distance_7,bids_limit_notional_7,bids_market_notional_7,bids_cancel_notional_7,bids_notional_7,asks_distance_7,asks_limit_notional_7,asks_market_notional_7,asks_cancel_notional_7,asks_notional_7,bids_distance_8,bids_limit_notional_8,bids_market_notional_8,bids_cancel_notional_8,bids_notional_8,asks_distance_8,asks_limit_notional_8,asks_market_notional_8,asks_cancel_notional_8,asks_notional_8,bids_distance_9,bids_limit_notional_9,bids_market_notional_9,bids_cancel_notional_9,bids_notional_9,asks_distance_9,asks_limit_notional_9,asks_market_notional_9,asks_cancel_notional_9,asks_notional_9,bids_distance_10,bids_limit_notional_10,bids_market_notional_10,bids_cancel_notional_10,bids_notional_10,asks_distance_10,asks_limit_notional_10,asks_market_notional_10,asks_cancel_notional_10,asks_notional_10,bids_distance_11,bids_limit_notional_11,bids_market_notional_11,bids_cancel_notional_11,bids_notional_11,asks_distance_11,asks_limit_notional_11,asks_market_notional_11,asks_cancel_notional_11,asks_notional_11,bids_distance_12,bids_limit_notional_12,bids_market_notional_12,bids_cancel_notional_12,bids_notional_12,asks_distance_12,asks_limit_notional_12,asks_market_notional_12,asks_cancel_notional_12,asks_notional_12,bids_distance_13,bids_limit_notional_13,bids_market_notional_13,bids_cancel_notional_13,bids_notional_13,asks_distance_13,asks_limit_notional_13,asks_market_notional_13,asks_cancel_notional_13,asks_notional_13,bids_distance_14,bids_limit_notional_14,bids_market_notional_14,bids_cancel_notional_14,bids_notional_14,asks_distance_14,asks_limit_notional_14,asks_market_notional_14,asks_cancel_notional_14,asks_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:33:59.055697+00:00,1.16205,0.0001,56936.467913,258248.957367,-0.000043,244337.5,1088.469971,242372.171875,876.869995,0.000043,3969.52002,0.0,697.549988,3271.969971,-0.000129,29571.470703,0.0,23587.300781,5984.169922,0.000904,4606.839844,0.0,0.0,4606.839844,-0.000301,28968.730469,0.0,28962.919922,5.81,0.001162,41899.851562,2464.080078,17953.589844,21482.179688,-0.000473,19662.660156,0.0,19644.429688,18.24,0.001248,614.330017,69.75,0.0,544.580017,-0.000731,37229.859375,0.0,17385.220703,19844.640625,0.001678,7196.200195,0.0,5192.410156,2003.790039,-0.000818,210722.640625,0.0,210554.28125,168.360001,0.002108,1979.650024,0.0,0.0,1979.650024,-0.000904,75917.0625,0.0,70002.617188,5914.439941,0.002194,5425.660156,0.0,5421.02002,4.65,-0.0009

In [9]:
# Optionally filter out "cancel" features/columns from dataframe

if (REMOVE_CANCEL_FEATURES):
    for crypto_ob_key in crypto_ob_df_dict:
        # Uses regular expressions to search for any string in the column name with "*cancel*"
        crypto_ob_df_dict[crypto_ob_key] = crypto_ob_df_dict[crypto_ob_key][crypto_ob_df_dict[crypto_ob_key].columns.drop(list(crypto_ob_df_dict[crypto_ob_key].filter(regex="cancel")))]

        if (TURN_ON_PRINT_DEBUG):
            print(f"\nDebug {crypto_ob_key} dataframe after filtering out cancels:")
            display(crypto_ob_df_dict[crypto_ob_key].shape)
            display(crypto_ob_df_dict[crypto_ob_key].head(n=1))


Debug BTC_5min dataframe after filtering out cancels:


(3367, 124)

,midpoint,spread,buys,sells,bids_distance_0,bids_limit_notional_0,bids_market_notional_0,bids_notional_0,asks_distance_0,asks_limit_notional_0,asks_market_notional_0,asks_notional_0,bids_distance_1,bids_limit_notional_1,bids_market_notional_1,bids_notional_1,asks_distance_1,asks_limit_notional_1,asks_market_notional_1,asks_notional_1,bids_distance_2,bids_limit_notional_2,bids_market_notional_2,bids_notional_2,asks_distance_2,asks_limit_notional_2,asks_market_notional_2,asks_notional_2,bids_distance_3,bids_limit_notional_3,bids_market_notional_3,bids_notional_3,asks_distance_3,asks_limit_notional_3,asks_market_notional_3,asks_notional_3,bids_distance_4,bids_limit_notional_4,bids_market_notional_4,bids_notional_4,asks_distance_4,asks_limit_notional_4,asks_market_notional_4,asks_notional_4,bids_distance_5,bids_limit_notional_5,bids_market_notional_5,bids_notional_5,asks_distance_5,asks_limit_notional_5,asks_market_notional_5,asks_notional_5,bids_distance_6,bids_limit_notional_6,bids_market_notional_6,bids_notional_6,asks_distance_6,asks_limit_notional_6,asks_market_notional_6,asks_notional_6,bids_distance_7,bids_limit_notional_7,bids_market_notional_7,bids_notional_7,asks_distance_7,asks_limit_notional_7,asks_market_notional_7,asks_notional_7,bids_distance_8,bids_limit_notional_8,bids_market_notional_8,bids_notional_8,asks_distance_8,asks_limit_notional_8,asks_market_notional_8,asks_notional_8,bids_distance_9,bids_limit_notional_9,bids_market_notional_9,bids_notional_9,asks_distance_9,asks_limit_notional_9,asks_market_notional_9,asks_notional_9,bids_distance_10,bids_limit_notional_10,bids_market_notional_10,bids_notional_10,asks_distance_10,asks_limit_notional_10,asks_market_notional_10,asks_notional_10,bids_distance_11,bids_limit_notional_11,bids_market_notional_11,bids_notional_11,asks_distance_11,asks_limit_notional_11,asks_market_notional_11,asks_notional_11,bids_distance_12,bids_limit_notional_12,bids_market_notional_12,bids_notional_12,asks_distance_12,asks_limit_notional_12,asks_market_notional_12,asks_notional_12,bids_distance_13,bids_limit_notional_13,bids_market_notional_13,bids_notional_13,asks_distance_13,asks_limit_notional_13,asks_market_notional_13,asks_notional_13,bids_distance_14,bids_limit_notional_14,bids_market_notional_14,bids_notional_14,asks_distance_14,asks_limit_notional_14,asks_market_notional_14,asks_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:41.122161+00:00,55899.995,0.01,1.199304e+07,1.348616e+07,-8.944544e-08,20307.839844,0.0,18997.929688,8.944544e-08,2208291.0,1986.449951,1098659.25,-0.000129,746.049988,0.0,746.049988,0.000033,345.220001,0.0,345.220001,-0.00013,1117.859985,0.0,1117.859985,0.000278,107.75,0.0,107.75,-0.000143,2794.600098,0.0,2794.600098,0.000345,27026.910156,0.0,581.559998,-0.000201,14100.320312,0.0,14100.320312,0.000364,283.829987,0.0,283.829987,-0.000201,5588.879883,0.0,5588.879883,0.000421,3511.439941,312.040009,2484.139893,-0.00024,187.220001,0.0,187.220001,0.0005,2164.22998,0.0,2164.22998,-0.000271,27942.429688,0.0,27942.429688,0.000573,19576.220703,0.0,19576.220703,-0.000387,14594.120117,0.0,13969.589844,0.000574,67001.023438,0.0,67001.023438,-0.000399,3362.449951,0.0,3362.449951,0.000586,52967.460938,0.0,64.040001,-0.000424,27044.119141,0.0,27044.119141,0.000592,105933.867188,0.0,126.389999,-0.000588,2793.360107,0.0,2793.360107,0.0006,53012.140625,0.0,107.980003,-0.000588,25698.880859,0.0,25698.880859,0.000638,64.040001,0.0,64.040001,-0.000629,2793.23999,0.0,2793.23999,0.000705,26541.439453,0.0,86.589996,-0.000727,500.0,0.0,500.0,0.000787,885565.5,0.0,2237.76001



Debug ETH_5min dataframe after filtering out cancels:


(3365, 124)

...

(17110, 124)

,midpoint,spread,buys,sells,bids_distance_0,bids_limit_notional_0,bids_market_notional_0,bids_notional_0,asks_distance_0,asks_limit_notional_0,asks_market_notional_0,asks_notional_0,bids_distance_1,bids_limit_notional_1,bids_market_notional_1,bids_notional_1,asks_distance_1,asks_limit_notional_1,asks_market_notional_1,asks_notional_1,bids_distance_2,bids_limit_notional_2,bids_market_notional_2,bids_notional_2,asks_distance_2,asks_limit_notional_2,asks_market_notional_2,asks_notional_2,bids_distance_3,bids_limit_notional_3,bids_market_notional_3,bids_notional_3,asks_distance_3,asks_limit_notional_3,asks_market_notional_3,asks_notional_3,bids_distance_4,bids_limit_notional_4,bids_market_notional_4,bids_notional_4,asks_distance_4,asks_limit_notional_4,asks_market_notional_4,asks_notional_4,bids_distance_5,bids_limit_notional_5,bids_market_notional_5,bids_notional_5,asks_distance_5,asks_limit_notional_5,asks_market_notional_5,asks_notional_5,bids_distance_6,bids_limit_notional_6,bids_market_notional_6,bids_notional_6,asks_distance_6,asks_limit_notional_6,asks_market_notional_6,asks_notional_6,bids_distance_7,bids_limit_notional_7,bids_market_notional_7,bids_notional_7,asks_distance_7,asks_limit_notional_7,asks_market_notional_7,asks_notional_7,bids_distance_8,bids_limit_notional_8,bids_market_notional_8,bids_notional_8,asks_distance_8,asks_limit_notional_8,asks_market_notional_8,asks_notional_8,bids_distance_9,bids_limit_notional_9,bids_market_notional_9,bids_notional_9,asks_distance_9,asks_limit_notional_9,asks_market_notional_9,asks_notional_9,bids_distance_10,bids_limit_notional_10,bids_market_notional_10,bids_notional_10,asks_distance_10,asks_limit_notional_10,asks_market_notional_10,asks_notional_10,bids_distance_11,bids_limit_notional_11,bids_market_notional_11,bids_notional_11,asks_distance_11,asks_limit_notional_11,asks_market_notional_11,asks_notional_11,bids_distance_12,bids_limit_notional_12,bids_market_notional_12,bids_notional_12,asks_distance_12,asks_limit_notional_12,asks_market_notional_12,asks_notional_12,bids_distance_13,bids_limit_notional_13,bids_market_notional_13,bids_notional_13,asks_distance_13,asks_limit_notional_13,asks_market_notional_13,asks_notional_13,bids_distance_14,bids_limit_notional_14,bids_market_notional_14,bids_notional_14,asks_distance_14,asks_limit_notional_14,asks_market_notional_14,asks_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:33:49.861733+00:00,1965.845,0.01,875154.482918,1.684774e+06,-0.000003,86711.492188,0.0,44445.929688,0.000003,20005.570312,20004.490234,1.08,-0.000155,9650.30957,0.0,7766.830078,0.000135,3234.25,0.0,3234.25,-0.00016,29679.5,0.0,196.550003,0.00014,20005.310547,0.0,20005.310547,-0.000257,9972.139648,0.0,9972.139648,0.000196,4350.029785,0.0,4350.029785,-0.000262,83933.03125,0.0,42644.878906,0.000242,5839.970215,0.0,3873.649902,-0.000277,34927.25,0.0,34927.25,0.000277,20005.439453,0.0,20005.439453,-0.000425,19519.410156,0.0,19519.410156,0.000415,3874.320068,0.0,3874.320068,-0.00043,14049.75,2032.810059,12016.94043,0.000455,731.830017,0.0,731.830017,-0.000455,514.400024,0.0,514.400024,0.000511,373.700012,0.0,373.700012,-0.000506,751.210022,0.0,751.210022,0.000567,196.699997,0.0,196.699997,-0.000532,2057.439941,0.0,2057.439941,0.000572,2059.709961,0.0,2059.709961,-0.000547,760.76001,0.0,760.76001,0.000588,3874.98999,0.0,3874.98999,-0.000593,4954.089844,0.0,4954.089844,0.000623,5901.209961,0.0,5901.209961,-0.000613,4995.359863,0.0,4995.359863,0.000628,178.289993,0.0,178.289993,-0.000674,1998.48999,0.0,1998.48999,0.000633,28512.160156,0.0,28512.160156



Debug ADA_1min dataframe after filtering out cancels:


(17109, 124)

,midpoint,spread,buys,sells,bids_distance_0,bids_limit_notional_0,bids_market_notional_0,bids_notional_0,asks_distance_0,asks_limit_notional_0,asks_market_notional_0,asks_notional_0,bids_distance_1,bids_limit_notional_1,bids_market_notional_1,bids_notional_1,asks_distance_1,asks_limit_notional_1,asks_market_notional_1,asks_notional_1,bids_distance_2,bids_limit_notional_2,bids_market_notional_2,bids_notional_2,asks_distance_2,asks_limit_notional_2,asks_market_notional_2,asks_notional_2,bids_distance_3,bids_limit_notional_3,bids_market_notional_3,bids_notional_3,asks_distance_3,asks_limit_notional_3,asks_market_notional_3,asks_notional_3,bids_distance_4,bids_limit_notional_4,bids_market_notional_4,bids_notional_4,asks_distance_4,asks_limit_notional_4,asks_market_notional_4,asks_notional_4,bids_distance_5,bids_limit_notional_5,bids_market_notional_5,bids_notional_5,asks_distance_5,asks_limit_notional_5,asks_market_notional_5,asks_notional_5,bids_distance_6,bids_limit_notional_6,bids_market_notional_6,bids_notional_6,asks_distance_6,asks_limit_notional_6,asks_market_notional_6,asks_notional_6,bids_distance_7,bids_limit_notional_7,bids_market_notional_7,bids_notional_7,asks_distance_7,asks_limit_notional_7,asks_market_notional_7,asks_notional_7,bids_distance_8,bids_limit_notional_8,bids_market_notional_8,bids_notional_8,asks_distance_8,asks_limit_notional_8,asks_market_notional_8,asks_notional_8,bids_distance_9,bids_limit_notional_9,bids_market_notional_9,bids_notional_9,asks_distance_9,asks_limit_notional_9,asks_market_notional_9,asks_notional_9,bids_distance_10,bids_limit_notional_10,bids_market_notional_10,bids_notional_10,asks_distance_10,asks_limit_notional_10,asks_market_notional_10,asks_notional_10,bids_distance_11,bids_limit_notional_11,bids_market_notional_11,bids_notional_11,asks_distance_11,asks_limit_notional_11,asks_market_notional_11,asks_notional_11,bids_distance_12,bids_limit_notional_12,bids_market_notional_12,bids_notional_12,asks_distance_12,asks_limit_notional_12,asks_market_notional_12,asks_notional_12,bids_distance_13,bids_limit_notional_13,bids_market_notional_13,bids_notional_13,asks_distance_13,asks_limit_notional_13,asks_market_notional_13,asks_notional_13,bids_distance_14,bids_limit_notional_14,bids_market_notional_14,bids_notional_14,asks_distance_14,asks_limit_notional_14,asks_market_notional_14,asks_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:33:59.055697+00:00,1.16205,0.0001,56936.467913,258248.957367,-0.000043,244337.5,1088.469971,876.869995,0.000043,3969.52002,0.0,3271.969971,-0.000129,29571.470703,0.0,5984.169922,0.000904,4606.839844,0.0,4606.839844,-0.000301,28968.730469,0.0,5.81,0.001162,41899.851562,2464.080078,21482.179688,-0.000473,19662.660156,0.0,18.24,0.001248,614.330017,69.75,544.580017,-0.000731,37229.859375,0.0,19844.640625,0.001678,7196.200195,0.0,2003.790039,-0.000818,210722.640625,0.0,168.360001,0.002108,1979.650024,0.0,1979.650024,-0.000904,75917.0625,0.0,5914.439941,0.002194,5425.660156,0.0,4.65,-0.00099,46004.519531,0.0,1227.089966,0.00228,11006.240234,0.0,10197.94043,-0.001162,14429.490234,0.0,5723.609863,0.002367,7925.560059,0.0,5500.439941,-0.001248,129133.539062,0.0,2724.550049,0.002453,10089.839844,0.0,7664.52002,-0.001334,24648.210938,0.0,7897.859863,0.003227,24917.769531,0.0,20063.380859,-0.00142,126310.15625,0.0,25.799999,0.004174,1000.0,0.0,1000.0,-0.001506,54058.519531,0.0,11387.959961,0.00426,1411.040039,0.0,1411.040039,-0.001592,119443.632812,0.0,37.220001,0.004776,3392.23999,0.0,3392.23999,-0.001678,139347.15625,0.0,26919.900391,0.005551,438.179993,0.0,433.51001


In [10]:
# Optionally filter out "market" features/columns from dataframe

if (REMOVE_MARKET_FEATURES):
    for crypto_ob_key in crypto_ob_df_dict:
        # Uses regular expressions to search for any string in the column name with "*market*"
        crypto_ob_df_dict[crypto_ob_key] = crypto_ob_df_dict[crypto_ob_key][crypto_ob_df_dict[crypto_ob_key].columns.drop(list(crypto_ob_df_dict[crypto_ob_key].filter(regex="market")))]

        if (TURN_ON_PRINT_DEBUG):
            print(f"\nDebug {crypto_ob_key} dataframe after filtering out market:")
            display(crypto_ob_df_dict[crypto_ob_key].shape)
            display(crypto_ob_df_dict[crypto_ob_key].head(n=1))            


Debug BTC_5min dataframe after filtering out market:


(3367, 94)

,midpoint,spread,buys,sells,bids_distance_0,bids_limit_notional_0,bids_notional_0,asks_distance_0,asks_limit_notional_0,asks_notional_0,bids_distance_1,bids_limit_notional_1,bids_notional_1,asks_distance_1,asks_limit_notional_1,asks_notional_1,bids_distance_2,bids_limit_notional_2,bids_notional_2,asks_distance_2,asks_limit_notional_2,asks_notional_2,bids_distance_3,bids_limit_notional_3,bids_notional_3,asks_distance_3,asks_limit_notional_3,asks_notional_3,bids_distance_4,bids_limit_notional_4,bids_notional_4,asks_distance_4,asks_limit_notional_4,asks_notional_4,bids_distance_5,bids_limit_notional_5,bids_notional_5,asks_distance_5,asks_limit_notional_5,asks_notional_5,bids_distance_6,bids_limit_notional_6,bids_notional_6,asks_distance_6,asks_limit_notional_6,asks_notional_6,bids_distance_7,bids_limit_notional_7,bids_notional_7,asks_distance_7,asks_limit_notional_7,asks_notional_7,bids_distance_8,bids_limit_notional_8,bids_notional_8,asks_distance_8,asks_limit_notional_8,asks_notional_8,bids_distance_9,bids_limit_notional_9,bids_notional_9,asks_distance_9,asks_limit_notional_9,asks_notional_9,bids_distance_10,bids_limit_notional_10,bids_notional_10,asks_distance_10,asks_limit_notional_10,asks_notional_10,bids_distance_11,bids_limit_notional_11,bids_notional_11,asks_distance_11,asks_limit_notional_11,asks_notional_11,bids_distance_12,bids_limit_notional_12,bids_notional_12,asks_distance_12,asks_limit_notional_12,asks_notional_12,bids_distance_13,bids_limit_notional_13,bids_notional_13,asks_distance_13,asks_limit_notional_13,asks_notional_13,bids_distance_14,bids_limit_notional_14,bids_notional_14,asks_distance_14,asks_limit_notional_14,asks_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:41.122161+00:00,55899.995,0.01,1.199304e+07,1.348616e+07,-8.944544e-08,20307.839844,18997.929688,8.944544e-08,2208291.0,1098659.25,-0.000129,746.049988,746.049988,0.000033,345.220001,345.220001,-0.00013,1117.859985,1117.859985,0.000278,107.75,107.75,-0.000143,2794.600098,2794.600098,0.000345,27026.910156,581.559998,-0.000201,14100.320312,14100.320312,0.000364,283.829987,283.829987,-0.000201,5588.879883,5588.879883,0.000421,3511.439941,2484.139893,-0.00024,187.220001,187.220001,0.0005,2164.22998,2164.22998,-0.000271,27942.429688,27942.429688,0.000573,19576.220703,19576.220703,-0.000387,14594.120117,13969.589844,0.000574,67001.023438,67001.023438,-0.000399,3362.449951,3362.449951,0.000586,52967.460938,64.040001,-0.000424,27044.119141,27044.119141,0.000592,105933.867188,126.389999,-0.000588,2793.360107,2793.360107,0.0006,53012.140625,107.980003,-0.000588,25698.880859,25698.880859,0.000638,64.040001,64.040001,-0.000629,2793.23999,2793.23999,0.000705,26541.439453,86.589996,-0.000727,500.0,500.0,0.000787,885565.5,2237.76001



Debug ETH_5min dataframe after filtering out market:


(3365, 94)

...

(17110, 94)

,midpoint,spread,buys,sells,bids_distance_0,bids_limit_notional_0,bids_notional_0,asks_distance_0,asks_limit_notional_0,asks_notional_0,bids_distance_1,bids_limit_notional_1,bids_notional_1,asks_distance_1,asks_limit_notional_1,asks_notional_1,bids_distance_2,bids_limit_notional_2,bids_notional_2,asks_distance_2,asks_limit_notional_2,asks_notional_2,bids_distance_3,bids_limit_notional_3,bids_notional_3,asks_distance_3,asks_limit_notional_3,asks_notional_3,bids_distance_4,bids_limit_notional_4,bids_notional_4,asks_distance_4,asks_limit_notional_4,asks_notional_4,bids_distance_5,bids_limit_notional_5,bids_notional_5,asks_distance_5,asks_limit_notional_5,asks_notional_5,bids_distance_6,bids_limit_notional_6,bids_notional_6,asks_distance_6,asks_limit_notional_6,asks_notional_6,bids_distance_7,bids_limit_notional_7,bids_notional_7,asks_distance_7,asks_limit_notional_7,asks_notional_7,bids_distance_8,bids_limit_notional_8,bids_notional_8,asks_distance_8,asks_limit_notional_8,asks_notional_8,bids_distance_9,bids_limit_notional_9,bids_notional_9,asks_distance_9,asks_limit_notional_9,asks_notional_9,bids_distance_10,bids_limit_notional_10,bids_notional_10,asks_distance_10,asks_limit_notional_10,asks_notional_10,bids_distance_11,bids_limit_notional_11,bids_notional_11,asks_distance_11,asks_limit_notional_11,asks_notional_11,bids_distance_12,bids_limit_notional_12,bids_notional_12,asks_distance_12,asks_limit_notional_12,asks_notional_12,bids_distance_13,bids_limit_notional_13,bids_notional_13,asks_distance_13,asks_limit_notional_13,asks_notional_13,bids_distance_14,bids_limit_notional_14,bids_notional_14,asks_distance_14,asks_limit_notional_14,asks_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:33:49.861733+00:00,1965.845,0.01,875154.482918,1.684774e+06,-0.000003,86711.492188,44445.929688,0.000003,20005.570312,1.08,-0.000155,9650.30957,7766.830078,0.000135,3234.25,3234.25,-0.00016,29679.5,196.550003,0.00014,20005.310547,20005.310547,-0.000257,9972.139648,9972.139648,0.000196,4350.029785,4350.029785,-0.000262,83933.03125,42644.878906,0.000242,5839.970215,3873.649902,-0.000277,34927.25,34927.25,0.000277,20005.439453,20005.439453,-0.000425,19519.410156,19519.410156,0.000415,3874.320068,3874.320068,-0.00043,14049.75,12016.94043,0.000455,731.830017,731.830017,-0.000455,514.400024,514.400024,0.000511,373.700012,373.700012,-0.000506,751.210022,751.210022,0.000567,196.699997,196.699997,-0.000532,2057.439941,2057.439941,0.000572,2059.709961,2059.709961,-0.000547,760.76001,760.76001,0.000588,3874.98999,3874.98999,-0.000593,4954.089844,4954.089844,0.000623,5901.209961,5901.209961,-0.000613,4995.359863,4995.359863,0.000628,178.289993,178.289993,-0.000674,1998.48999,1998.48999,0.000633,28512.160156,28512.160156



Debug ADA_1min dataframe after filtering out market:


(17109, 94)

,midpoint,spread,buys,sells,bids_distance_0,bids_limit_notional_0,bids_notional_0,asks_distance_0,asks_limit_notional_0,asks_notional_0,bids_distance_1,bids_limit_notional_1,bids_notional_1,asks_distance_1,asks_limit_notional_1,asks_notional_1,bids_distance_2,bids_limit_notional_2,bids_notional_2,asks_distance_2,asks_limit_notional_2,asks_notional_2,bids_distance_3,bids_limit_notional_3,bids_notional_3,asks_distance_3,asks_limit_notional_3,asks_notional_3,bids_distance_4,bids_limit_notional_4,bids_notional_4,asks_distance_4,asks_limit_notional_4,asks_notional_4,bids_distance_5,bids_limit_notional_5,bids_notional_5,asks_distance_5,asks_limit_notional_5,asks_notional_5,bids_distance_6,bids_limit_notional_6,bids_notional_6,asks_distance_6,asks_limit_notional_6,asks_notional_6,bids_distance_7,bids_limit_notional_7,bids_notional_7,asks_distance_7,asks_limit_notional_7,asks_notional_7,bids_distance_8,bids_limit_notional_8,bids_notional_8,asks_distance_8,asks_limit_notional_8,asks_notional_8,bids_distance_9,bids_limit_notional_9,bids_notional_9,asks_distance_9,asks_limit_notional_9,asks_notional_9,bids_distance_10,bids_limit_notional_10,bids_notional_10,asks_distance_10,asks_limit_notional_10,asks_notional_10,bids_distance_11,bids_limit_notional_11,bids_notional_11,asks_distance_11,asks_limit_notional_11,asks_notional_11,bids_distance_12,bids_limit_notional_12,bids_notional_12,asks_distance_12,asks_limit_notional_12,asks_notional_12,bids_distance_13,bids_limit_notional_13,bids_notional_13,asks_distance_13,asks_limit_notional_13,asks_notional_13,bids_distance_14,bids_limit_notional_14,bids_notional_14,asks_distance_14,asks_limit_notional_14,asks_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:33:59.055697+00:00,1.16205,0.0001,56936.467913,258248.957367,-0.000043,244337.5,876.869995,0.000043,3969.52002,3271.969971,-0.000129,29571.470703,5984.169922,0.000904,4606.839844,4606.839844,-0.000301,28968.730469,5.81,0.001162,41899.851562,21482.179688,-0.000473,19662.660156,18.24,0.001248,614.330017,544.580017,-0.000731,37229.859375,19844.640625,0.001678,7196.200195,2003.790039,-0.000818,210722.640625,168.360001,0.002108,1979.650024,1979.650024,-0.000904,75917.0625,5914.439941,0.002194,5425.660156,4.65,-0.00099,46004.519531,1227.089966,0.00228,11006.240234,10197.94043,-0.001162,14429.490234,5723.609863,0.002367,7925.560059,5500.439941,-0.001248,129133.539062,2724.550049,0.002453,10089.839844,7664.52002,-0.001334,24648.210938,7897.859863,0.003227,24917.769531,20063.380859,-0.00142,126310.15625,25.799999,0.004174,1000.0,1000.0,-0.001506,54058.519531,11387.959961,0.00426,1411.040039,1411.040039,-0.001592,119443.632812,37.220001,0.004776,3392.23999,3392.23999,-0.001678,139347.15625,26919.900391,0.005551,438.179993,433.51001


In [11]:
# Optionally filter out final "notional" features/columns from dataframe

if (REMOVE_MARKET_FEATURES):
    for crypto_ob_key in crypto_ob_df_dict:
        for bid_ask_str in bid_ask_str_lst:
            final_notional_level_str = bid_ask_str + "_notional_"
            # Uses regular expressions to search for any string in the column name with "bids_notional*" and "asks_notional*"
            crypto_ob_df_dict[crypto_ob_key] = crypto_ob_df_dict[crypto_ob_key][crypto_ob_df_dict[crypto_ob_key].columns.drop(list(crypto_ob_df_dict[crypto_ob_key].filter(regex=final_notional_level_str)))]
            #print (f"{final_notional_level_str}")
            
        if (TURN_ON_PRINT_DEBUG):
            print(f"\nDebug {crypto_ob_key} dataframe after filtering out final notional:")
            display(crypto_ob_df_dict[crypto_ob_key].shape)
            display(crypto_ob_df_dict[crypto_ob_key].head(n=1)) 


Debug BTC_5min dataframe after filtering out final notional:


(3367, 64)

,midpoint,spread,buys,sells,bids_distance_0,bids_limit_notional_0,asks_distance_0,asks_limit_notional_0,bids_distance_1,bids_limit_notional_1,asks_distance_1,asks_limit_notional_1,bids_distance_2,bids_limit_notional_2,asks_distance_2,asks_limit_notional_2,bids_distance_3,bids_limit_notional_3,asks_distance_3,asks_limit_notional_3,bids_distance_4,bids_limit_notional_4,asks_distance_4,asks_limit_notional_4,bids_distance_5,bids_limit_notional_5,asks_distance_5,asks_limit_notional_5,bids_distance_6,bids_limit_notional_6,asks_distance_6,asks_limit_notional_6,bids_distance_7,bids_limit_notional_7,asks_distance_7,asks_limit_notional_7,bids_distance_8,bids_limit_notional_8,asks_distance_8,asks_limit_notional_8,bids_distance_9,bids_limit_notional_9,asks_distance_9,asks_limit_notional_9,bids_distance_10,bids_limit_notional_10,asks_distance_10,asks_limit_notional_10,bids_distance_11,bids_limit_notional_11,asks_distance_11,asks_limit_notional_11,bids_distance_12,bids_limit_notional_12,asks_distance_12,asks_limit_notional_12,bids_distance_13,bids_limit_notional_13,asks_distance_13,asks_limit_notional_13,bids_distance_14,bids_limit_notional_14,asks_distance_14,asks_limit_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:41.122161+00:00,55899.995,0.01,1.199304e+07,1.348616e+07,-8.944544e-08,20307.839844,8.944544e-08,2208291.0,-0.000129,746.049988,0.000033,345.220001,-0.00013,1117.859985,0.000278,107.75,-0.000143,2794.600098,0.000345,27026.910156,-0.000201,14100.320312,0.000364,283.829987,-0.000201,5588.879883,0.000421,3511.439941,-0.00024,187.220001,0.0005,2164.22998,-0.000271,27942.429688,0.000573,19576.220703,-0.000387,14594.120117,0.000574,67001.023438,-0.000399,3362.449951,0.000586,52967.460938,-0.000424,27044.119141,0.000592,105933.867188,-0.000588,2793.360107,0.0006,53012.140625,-0.000588,25698.880859,0.000638,64.040001,-0.000629,2793.23999,0.000705,26541.439453,-0.000727,500.0,0.000787,885565.5



Debug ETH_5min dataframe after filtering out final notional:


(3365, 64)

...

(17110, 64)

,midpoint,spread,buys,sells,bids_distance_0,bids_limit_notional_0,asks_distance_0,asks_limit_notional_0,bids_distance_1,bids_limit_notional_1,asks_distance_1,asks_limit_notional_1,bids_distance_2,bids_limit_notional_2,asks_distance_2,asks_limit_notional_2,bids_distance_3,bids_limit_notional_3,asks_distance_3,asks_limit_notional_3,bids_distance_4,bids_limit_notional_4,asks_distance_4,asks_limit_notional_4,bids_distance_5,bids_limit_notional_5,asks_distance_5,asks_limit_notional_5,bids_distance_6,bids_limit_notional_6,asks_distance_6,asks_limit_notional_6,bids_distance_7,bids_limit_notional_7,asks_distance_7,asks_limit_notional_7,bids_distance_8,bids_limit_notional_8,asks_distance_8,asks_limit_notional_8,bids_distance_9,bids_limit_notional_9,asks_distance_9,asks_limit_notional_9,bids_distance_10,bids_limit_notional_10,asks_distance_10,asks_limit_notional_10,bids_distance_11,bids_limit_notional_11,asks_distance_11,asks_limit_notional_11,bids_distance_12,bids_limit_notional_12,asks_distance_12,asks_limit_notional_12,bids_distance_13,bids_limit_notional_13,asks_distance_13,asks_limit_notional_13,bids_distance_14,bids_limit_notional_14,asks_distance_14,asks_limit_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:33:49.861733+00:00,1965.845,0.01,875154.482918,1.684774e+06,-0.000003,86711.492188,0.000003,20005.570312,-0.000155,9650.30957,0.000135,3234.25,-0.00016,29679.5,0.00014,20005.310547,-0.000257,9972.139648,0.000196,4350.029785,-0.000262,83933.03125,0.000242,5839.970215,-0.000277,34927.25,0.000277,20005.439453,-0.000425,19519.410156,0.000415,3874.320068,-0.00043,14049.75,0.000455,731.830017,-0.000455,514.400024,0.000511,373.700012,-0.000506,751.210022,0.000567,196.699997,-0.000532,2057.439941,0.000572,2059.709961,-0.000547,760.76001,0.000588,3874.98999,-0.000593,4954.089844,0.000623,5901.209961,-0.000613,4995.359863,0.000628,178.289993,-0.000674,1998.48999,0.000633,28512.160156



Debug ADA_1min dataframe after filtering out final notional:


(17109, 64)

,midpoint,spread,buys,sells,bids_distance_0,bids_limit_notional_0,asks_distance_0,asks_limit_notional_0,bids_distance_1,bids_limit_notional_1,asks_distance_1,asks_limit_notional_1,bids_distance_2,bids_limit_notional_2,asks_distance_2,asks_limit_notional_2,bids_distance_3,bids_limit_notional_3,asks_distance_3,asks_limit_notional_3,bids_distance_4,bids_limit_notional_4,asks_distance_4,asks_limit_notional_4,bids_distance_5,bids_limit_notional_5,asks_distance_5,asks_limit_notional_5,bids_distance_6,bids_limit_notional_6,asks_distance_6,asks_limit_notional_6,bids_distance_7,bids_limit_notional_7,asks_distance_7,asks_limit_notional_7,bids_distance_8,bids_limit_notional_8,asks_distance_8,asks_limit_notional_8,bids_distance_9,bids_limit_notional_9,asks_distance_9,asks_limit_notional_9,bids_distance_10,bids_limit_notional_10,asks_distance_10,asks_limit_notional_10,bids_distance_11,bids_limit_notional_11,asks_distance_11,asks_limit_notional_11,bids_distance_12,bids_limit_notional_12,asks_distance_12,asks_limit_notional_12,bids_distance_13,bids_limit_notional_13,asks_distance_13,asks_limit_notional_13,bids_distance_14,bids_limit_notional_14,asks_distance_14,asks_limit_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:33:59.055697+00:00,1.16205,0.0001,56936.467913,258248.957367,-0.000043,244337.5,0.000043,3969.52002,-0.000129,29571.470703,0.000904,4606.839844,-0.000301,28968.730469,0.001162,41899.851562,-0.000473,19662.660156,0.001248,614.330017,-0.000731,37229.859375,0.001678,7196.200195,-0.000818,210722.640625,0.002108,1979.650024,-0.000904,75917.0625,0.002194,5425.660156,-0.00099,46004.519531,0.00228,11006.240234,-0.001162,14429.490234,0.002367,7925.560059,-0.001248,129133.539062,0.002453,10089.839844,-0.001334,24648.210938,0.003227,24917.769531,-0.00142,126310.15625,0.004174,1000.0,-0.001506,54058.519531,0.00426,1411.040039,-0.001592,119443.632812,0.004776,3392.23999,-0.001678,139347.15625,0.005551,438.179993


In [12]:
# Filter # levels from dataframe 
# Select only the levels interested: From Level_0 --> NUM_ORDER_BOOK_LEVELS-1, drop all other levels

if (NUM_ORDER_BOOK_LEVELS < MAX_ORDER_BOOK_LEVELS):
    for crypto_ob_key in crypto_ob_df_dict:
        # Uses regular expressions to search for any string in the column name with strings (e.g. _4, _5, _6, ...)
        for remove_level_num in range(NUM_ORDER_BOOK_LEVELS, MAX_ORDER_BOOK_LEVELS):   # Note, 1st level = level_0
            remove_level_str = "_" + str(remove_level_num)
            crypto_ob_df_dict[crypto_ob_key] = crypto_ob_df_dict[crypto_ob_key][crypto_ob_df_dict[crypto_ob_key].columns.drop(list(crypto_ob_df_dict[crypto_ob_key].filter(regex=remove_level_str)))]

        if (TURN_ON_PRINT_DEBUG):
            print(f"\nDebug {crypto_ob_key} dataframe after filtering levels:")
            display(crypto_ob_df_dict[crypto_ob_key].shape)
            display(crypto_ob_df_dict[crypto_ob_key].head(n=1))                

In [13]:
# Optionally, add actual prices at order book levels
# The provided order book data does not contain the actual price of the asset at each level, but instead provides the "delta/difference in %" (or distance) between the actual price and the mid price
# This will derive the actual price (mid_price + distance) and create a new column/feature in the dataframe

if (ADD_ABSOLUTE_PRICE_LEVELS):
    for crypto_ob_key in crypto_ob_df_dict:
        for level_num in range(NUM_ORDER_BOOK_LEVELS):
            for bid_ask_str in bid_ask_str_lst:
                distance_level_str = bid_ask_str + "_distance_" + str(level_num)   # Existing delta/distance price column name (e.g. bids_distance_0, bids_distance_1, ..., asks_distance_0,...)
                price_level_str = bid_ask_str + "_price_" + str(level_num)     # New column will be added to dataframe with actual prices (e.g. bids_price_0, bids_price_1, ..., asks_price_0, ...)
                
                # The distance is in % difference between the midpoint price and current level price
                # To compute the price = midpoint * (1 + %/distance)
                price = crypto_ob_df_dict[crypto_ob_key]["midpoint"] * (1 + crypto_ob_df_dict[crypto_ob_key][distance_level_str])
                
                # Want to insert the price column at a specific level right next to the price distance column for easy viewing (e.g. "bids_distance_0", "bids_price_0", ...)
                distance_column_index = crypto_ob_df_dict[crypto_ob_key].columns.get_loc(distance_level_str)
                crypto_ob_df_dict[crypto_ob_key].insert(distance_column_index+1, price_level_str, price)        

        if (TURN_ON_PRINT_DEBUG):
            print(f"\nDebug {crypto_ob_key} dataframe after adding price level columns:")
            display(crypto_ob_df_dict[crypto_ob_key].shape)
            display(crypto_ob_df_dict[crypto_ob_key].head(n=1))                


Debug BTC_5min dataframe after adding price level columns:


(3367, 94)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_notional_0,asks_distance_0,asks_price_0,asks_limit_notional_0,bids_distance_1,bids_price_1,bids_limit_notional_1,asks_distance_1,asks_price_1,asks_limit_notional_1,bids_distance_2,bids_price_2,bids_limit_notional_2,asks_distance_2,asks_price_2,asks_limit_notional_2,bids_distance_3,bids_price_3,bids_limit_notional_3,asks_distance_3,asks_price_3,asks_limit_notional_3,bids_distance_4,bids_price_4,bids_limit_notional_4,asks_distance_4,asks_price_4,asks_limit_notional_4,bids_distance_5,bids_price_5,bids_limit_notional_5,asks_distance_5,asks_price_5,asks_limit_notional_5,bids_distance_6,bids_price_6,bids_limit_notional_6,asks_distance_6,asks_price_6,asks_limit_notional_6,bids_distance_7,bids_price_7,bids_limit_notional_7,asks_distance_7,asks_price_7,asks_limit_notional_7,bids_distance_8,bids_price_8,bids_limit_notional_8,asks_distance_8,asks_price_8,asks_limit_notional_8,bids_distance_9,bids_price_9,bids_limit_notional_9,asks_distance_9,asks_price_9,asks_limit_notional_9,bids_distance_10,bids_price_10,bids_limit_notional_10,asks_distance_10,asks_price_10,asks_limit_notional_10,bids_distance_11,bids_price_11,bids_limit_notional_11,asks_distance_11,asks_price_11,asks_limit_notional_11,bids_distance_12,bids_price_12,bids_limit_notional_12,asks_distance_12,asks_price_12,asks_limit_notional_12,bids_distance_13,bids_price_13,bids_limit_notional_13,asks_distance_13,asks_price_13,asks_limit_notional_13,bids_distance_14,bids_price_14,bids_limit_notional_14,asks_distance_14,asks_price_14,asks_limit_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:41.122161+00:00,55899.995,0.01,1.199304e+07,1.348616e+07,-8.944544e-08,55899.99,20307.839844,8.944544e-08,55900.0,2208291.0,-0.000129,55892.76,746.049988,0.000033,55901.84,345.220001,-0.00013,55892.75,1117.859985,0.000278,55915.510001,107.75,-0.000143,55892.02,2794.600098,0.000345,55919.3,27026.910156,-0.000201,55888.78,14100.320312,0.000364,55920.36,283.829987,-0.000201,55888.77,5588.879883,0.000421,55923.54,3511.439941,-0.00024,55886.56,187.220001,0.0005,55927.95,2164.22998,-0.000271,55884.86,27942.429688,0.000573,55932.05,19576.220703,-0.000387,55878.379999,14594.120117,0.000574,55932.059999,67001.023438,-0.000399,55877.71,3362.449951,0.000586,55932.76,52967.460938,-0.000424,55876.28,27044.119141,0.000592,55933.090001,105933.867188,-0.000588,55867.130001,2793.360107,0.0006,55933.530001,53012.140625,-0.000588,55867.119998,25698.880859,0.000638,55935.640001,64.040001,-0.000629,55864.830001,2793.23999,0.000705,55939.390001,26541.439453,-0.000727,55859.34,500.0,0.000787,55944.0,885565.5



Debug ETH_5min dataframe after adding price level columns:


(3365, 94)

...

(17110, 94)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_notional_0,asks_distance_0,asks_price_0,asks_limit_notional_0,bids_distance_1,bids_price_1,bids_limit_notional_1,asks_distance_1,asks_price_1,asks_limit_notional_1,bids_distance_2,bids_price_2,bids_limit_notional_2,asks_distance_2,asks_price_2,asks_limit_notional_2,bids_distance_3,bids_price_3,bids_limit_notional_3,asks_distance_3,asks_price_3,asks_limit_notional_3,bids_distance_4,bids_price_4,bids_limit_notional_4,asks_distance_4,asks_price_4,asks_limit_notional_4,bids_distance_5,bids_price_5,bids_limit_notional_5,asks_distance_5,asks_price_5,asks_limit_notional_5,bids_distance_6,bids_price_6,bids_limit_notional_6,asks_distance_6,asks_price_6,asks_limit_notional_6,bids_distance_7,bids_price_7,bids_limit_notional_7,asks_distance_7,asks_price_7,asks_limit_notional_7,bids_distance_8,bids_price_8,bids_limit_notional_8,asks_distance_8,asks_price_8,asks_limit_notional_8,bids_distance_9,bids_price_9,bids_limit_notional_9,asks_distance_9,asks_price_9,asks_limit_notional_9,bids_distance_10,bids_price_10,bids_limit_notional_10,asks_distance_10,asks_price_10,asks_limit_notional_10,bids_distance_11,bids_price_11,bids_limit_notional_11,asks_distance_11,asks_price_11,asks_limit_notional_11,bids_distance_12,bids_price_12,bids_limit_notional_12,asks_distance_12,asks_price_12,asks_limit_notional_12,bids_distance_13,bids_price_13,bids_limit_notional_13,asks_distance_13,asks_price_13,asks_limit_notional_13,bids_distance_14,bids_price_14,bids_limit_notional_14,asks_distance_14,asks_price_14,asks_limit_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:33:49.861733+00:00,1965.845,0.01,875154.482918,1.684774e+06,-0.000003,1965.84,86711.492188,0.000003,1965.85,20005.570312,-0.000155,1965.54,9650.30957,0.000135,1966.11,3234.25,-0.00016,1965.53,29679.5,0.00014,1966.12,20005.310547,-0.000257,1965.34,9972.139648,0.000196,1966.23,4350.029785,-0.000262,1965.33,83933.03125,0.000242,1966.32,5839.970215,-0.000277,1965.3,34927.25,0.000277,1966.39,20005.439453,-0.000425,1965.01,19519.410156,0.000415,1966.66,3874.320068,-0.00043,1965.0,14049.75,0.000455,1966.74,731.830017,-0.000455,1964.95,514.400024,0.000511,1966.85,373.700012,-0.000506,1964.85,751.210022,0.000567,1966.96,196.699997,-0.000532,1964.8,2057.439941,0.000572,1966.97,2059.709961,-0.000547,1964.77,760.76001,0.000588,1967.0,3874.98999,-0.000593,1964.68,4954.089844,0.000623,1967.07,5901.209961,-0.000613,1964.64,4995.359863,0.000628,1967.08,178.289993,-0.000674,1964.52,1998.48999,0.000633,1967.09,28512.160156



Debug ADA_1min dataframe after adding price level columns:


(17109, 94)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_notional_0,asks_distance_0,asks_price_0,asks_limit_notional_0,bids_distance_1,bids_price_1,bids_limit_notional_1,asks_distance_1,asks_price_1,asks_limit_notional_1,bids_distance_2,bids_price_2,bids_limit_notional_2,asks_distance_2,asks_price_2,asks_limit_notional_2,bids_distance_3,bids_price_3,bids_limit_notional_3,asks_distance_3,asks_price_3,asks_limit_notional_3,bids_distance_4,bids_price_4,bids_limit_notional_4,asks_distance_4,asks_price_4,asks_limit_notional_4,bids_distance_5,bids_price_5,bids_limit_notional_5,asks_distance_5,asks_price_5,asks_limit_notional_5,bids_distance_6,bids_price_6,bids_limit_notional_6,asks_distance_6,asks_price_6,asks_limit_notional_6,bids_distance_7,bids_price_7,bids_limit_notional_7,asks_distance_7,asks_price_7,asks_limit_notional_7,bids_distance_8,bids_price_8,bids_limit_notional_8,asks_distance_8,asks_price_8,asks_limit_notional_8,bids_distance_9,bids_price_9,bids_limit_notional_9,asks_distance_9,asks_price_9,asks_limit_notional_9,bids_distance_10,bids_price_10,bids_limit_notional_10,asks_distance_10,asks_price_10,asks_limit_notional_10,bids_distance_11,bids_price_11,bids_limit_notional_11,asks_distance_11,asks_price_11,asks_limit_notional_11,bids_distance_12,bids_price_12,bids_limit_notional_12,asks_distance_12,asks_price_12,asks_limit_notional_12,bids_distance_13,bids_price_13,bids_limit_notional_13,asks_distance_13,asks_price_13,asks_limit_notional_13,bids_distance_14,bids_price_14,bids_limit_notional_14,asks_distance_14,asks_price_14,asks_limit_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:33:59.055697+00:00,1.16205,0.0001,56936.467913,258248.957367,-0.000043,1.162,244337.5,0.000043,1.1621,3969.52002,-0.000129,1.1619,29571.470703,0.000904,1.1631,4606.839844,-0.000301,1.1617,28968.730469,0.001162,1.1634,41899.851562,-0.000473,1.1615,19662.660156,0.001248,1.1635,614.330017,-0.000731,1.1612,37229.859375,0.001678,1.164,7196.200195,-0.000818,1.1611,210722.640625,0.002108,1.1645,1979.650024,-0.000904,1.161,75917.0625,0.002194,1.1646,5425.660156,-0.00099,1.1609,46004.519531,0.00228,1.1647,11006.240234,-0.001162,1.1607,14429.490234,0.002367,1.1648,7925.560059,-0.001248,1.1606,129133.539062,0.002453,1.1649,10089.839844,-0.001334,1.1605,24648.210938,0.003227,1.1658,24917.769531,-0.00142,1.1604,126310.15625,0.004174,1.1669,1000.0,-0.001506,1.1603,54058.519531,0.00426,1.167,1411.040039,-0.001592,1.1602,119443.632812,0.004776,1.1676,3392.23999,-0.001678,1.1601,139347.15625,0.005551,1.1685,438.179993


In [14]:
# Optionally, filter out price distance features (e.g. bids_distance_0, bids_distance_1, ..., asks_distance_0,...)

if (REMOVE_PRICE_DISTANCES):
    for crypto_ob_key in crypto_ob_df_dict:
        # Uses regular expressions to search for any string in the column name with "*distance*"
        crypto_ob_df_dict[crypto_ob_key] = crypto_ob_df_dict[crypto_ob_key][crypto_ob_df_dict[crypto_ob_key].columns.drop(list(crypto_ob_df_dict[crypto_ob_key].filter(regex="_distance_")))]
        
        if (TURN_ON_PRINT_DEBUG):
            print(f"\nDebug {crypto_ob_key} dataframe after dropping distance column:")
            display(crypto_ob_df_dict[crypto_ob_key].shape)
            display(crypto_ob_df_dict[crypto_ob_key].head(n=1))        

In [15]:
# Add the quantity at each level
# The downloaded crypo order book provides the "notional" value which equals the price (at that level) * the quantity (at the level)
# Derive the quantiy by computing: notional / price

# There are 4 notional fields provided at each level: notional, limit_notional, market_notional, and cancel_notional
# where "notional = limit_notional - market_notional - cancel_notional"
notional_str_lst = ["_limit_"]
if (not REMOVE_MARKET_FEATURES): 
    notional_str_lst.append("_market_")
if (not REMOVE_CANCEL_FEATURES): 
    notional_str_lst.append("_cancel_")
if (not REMOVE_FINAL_NOTIONAL): 
    notional_str_lst.append("_")    # For the "notional" column by itself

for crypto_ob_key in crypto_ob_df_dict:
    for level_num in range(NUM_ORDER_BOOK_LEVELS):
        for bid_ask_str in bid_ask_str_lst:
            for notional_str in notional_str_lst:
                notional_level_str = bid_ask_str + notional_str + "notional" + "_" + str(level_num)
                price_level_str = bid_ask_str + "_price" + "_" + str(level_num)
                quantity_level_str = bid_ask_str + notional_str + "quantity" + "_" + str(level_num)
                
                # quantity = notional / price
                quantity = crypto_ob_df_dict[crypto_ob_key][notional_level_str] / crypto_ob_df_dict[crypto_ob_key][price_level_str]
                
                # Want to insert the quantity column at a specific level right before the notional column for easy viewing (e.g. "bids_quantity_0", "bids_notional_0", ...)
                notional_column_index = crypto_ob_df_dict[crypto_ob_key].columns.get_loc(notional_level_str)
                crypto_ob_df_dict[crypto_ob_key].insert(notional_column_index, quantity_level_str, quantity)   
                #print(f"{notional_level_str}, {price_level_str}, {quantity_level_str}")
                
    if (TURN_ON_PRINT_DEBUG):
        print(f"\nDebug {crypto_ob_key} dataframe after adding the quantity feature column:")
        display(crypto_ob_df_dict[crypto_ob_key].shape)
        display(crypto_ob_df_dict[crypto_ob_key].head(n=1))            


Debug BTC_5min dataframe after adding the quantity feature column:


(3367, 124)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_quantity_0,bids_limit_notional_0,asks_distance_0,asks_price_0,asks_limit_quantity_0,asks_limit_notional_0,bids_distance_1,bids_price_1,bids_limit_quantity_1,bids_limit_notional_1,asks_distance_1,asks_price_1,asks_limit_quantity_1,asks_limit_notional_1,bids_distance_2,bids_price_2,bids_limit_quantity_2,bids_limit_notional_2,asks_distance_2,asks_price_2,asks_limit_quantity_2,asks_limit_notional_2,bids_distance_3,bids_price_3,bids_limit_quantity_3,bids_limit_notional_3,asks_distance_3,asks_price_3,asks_limit_quantity_3,asks_limit_notional_3,bids_distance_4,bids_price_4,bids_limit_quantity_4,bids_limit_notional_4,asks_distance_4,asks_price_4,asks_limit_quantity_4,asks_limit_notional_4,bids_distance_5,bids_price_5,bids_limit_quantity_5,bids_limit_notional_5,asks_distance_5,asks_price_5,asks_limit_quantity_5,asks_limit_notional_5,bids_distance_6,bids_price_6,bids_limit_quantity_6,bids_limit_notional_6,asks_distance_6,asks_price_6,asks_limit_quantity_6,asks_limit_notional_6,bids_distance_7,bids_price_7,bids_limit_quantity_7,bids_limit_notional_7,asks_distance_7,asks_price_7,asks_limit_quantity_7,asks_limit_notional_7,bids_distance_8,bids_price_8,bids_limit_quantity_8,bids_limit_notional_8,asks_distance_8,asks_price_8,asks_limit_quantity_8,asks_limit_notional_8,bids_distance_9,bids_price_9,bids_limit_quantity_9,bids_limit_notional_9,asks_distance_9,asks_price_9,asks_limit_quantity_9,asks_limit_notional_9,bids_distance_10,bids_price_10,bids_limit_quantity_10,bids_limit_notional_10,asks_distance_10,asks_price_10,asks_limit_quantity_10,asks_limit_notional_10,bids_distance_11,bids_price_11,bids_limit_quantity_11,bids_limit_notional_11,asks_distance_11,asks_price_11,asks_limit_quantity_11,asks_limit_notional_11,bids_distance_12,bids_price_12,bids_limit_quantity_12,bids_limit_notional_12,asks_distance_12,asks_price_12,asks_limit_quantity_12,asks_limit_notional_12,bids_distance_13,bids_price_13,bids_limit_quantity_13,bids_limit_notional_13,asks_distance_13,asks_price_13,asks_limit_quantity_13,asks_limit_notional_13,bids_distance_14,bids_price_14,bids_limit_quantity_14,bids_limit_notional_14,asks_distance_14,asks_price_14,asks_limit_quantity_14,asks_limit_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:41.122161+00:00,55899.995,0.01,1.199304e+07,1.348616e+07,-8.944544e-08,55899.99,0.363289,20307.839844,8.944544e-08,55900.0,39.504311,2208291.0,-0.000129,55892.76,0.013348,746.049988,0.000033,55901.84,0.006175,345.220001,-0.00013,55892.75,0.02,1117.859985,0.000278,55915.510001,0.001927,107.75,-0.000143,55892.02,0.05,2794.600098,0.000345,55919.3,0.48332,27026.910156,-0.000201,55888.78,0.252293,14100.320312,0.000364,55920.36,0.005076,283.829987,-0.000201,55888.77,0.1,5588.879883,0.000421,55923.54,0.06279,3511.439941,-0.00024,55886.56,0.00335,187.220001,0.0005,55927.95,0.038697,2164.22998,-0.000271,55884.86,0.5,27942.429688,0.000573,55932.05,0.35,19576.220703,-0.000387,55878.379999,0.261177,14594.120117,0.000574,55932.059999,1.1979,67001.023438,-0.000399,55877.71,0.060175,3362.449951,0.000586,55932.76,0.946985,52967.460938,-0.000424,55876.28,0.484,27044.119141,0.000592,55933.090001,1.893939,105933.867188,-0.000588,55867.130001,0.05,2793.360107,0.0006,55933.530001,0.94777,53012.140625,-0.000588,55867.119998,0.46,25698.880859,0.000638,55935.640001,0.001145,64.040001,-0.000629,55864.830001,0.05,2793.23999,0.000705,55939.390001,0.474468,26541.439453,-0.000727,55859.34,0.008951,500.0,0.000787,55944.0,15.829499,885565.5



Debug ETH_5min dataframe after adding the quantity feature column:


(3365, 124)

...

(17110, 124)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_quantity_0,bids_limit_notional_0,asks_distance_0,asks_price_0,asks_limit_quantity_0,asks_limit_notional_0,bids_distance_1,bids_price_1,bids_limit_quantity_1,bids_limit_notional_1,asks_distance_1,asks_price_1,asks_limit_quantity_1,asks_limit_notional_1,bids_distance_2,bids_price_2,bids_limit_quantity_2,bids_limit_notional_2,asks_distance_2,asks_price_2,asks_limit_quantity_2,asks_limit_notional_2,bids_distance_3,bids_price_3,bids_limit_quantity_3,bids_limit_notional_3,asks_distance_3,asks_price_3,asks_limit_quantity_3,asks_limit_notional_3,bids_distance_4,bids_price_4,bids_limit_quantity_4,bids_limit_notional_4,asks_distance_4,asks_price_4,asks_limit_quantity_4,asks_limit_notional_4,bids_distance_5,bids_price_5,bids_limit_quantity_5,bids_limit_notional_5,asks_distance_5,asks_price_5,asks_limit_quantity_5,asks_limit_notional_5,bids_distance_6,bids_price_6,bids_limit_quantity_6,bids_limit_notional_6,asks_distance_6,asks_price_6,asks_limit_quantity_6,asks_limit_notional_6,bids_distance_7,bids_price_7,bids_limit_quantity_7,bids_limit_notional_7,asks_distance_7,asks_price_7,asks_limit_quantity_7,asks_limit_notional_7,bids_distance_8,bids_price_8,bids_limit_quantity_8,bids_limit_notional_8,asks_distance_8,asks_price_8,asks_limit_quantity_8,asks_limit_notional_8,bids_distance_9,bids_price_9,bids_limit_quantity_9,bids_limit_notional_9,asks_distance_9,asks_price_9,asks_limit_quantity_9,asks_limit_notional_9,bids_distance_10,bids_price_10,bids_limit_quantity_10,bids_limit_notional_10,asks_distance_10,asks_price_10,asks_limit_quantity_10,asks_limit_notional_10,bids_distance_11,bids_price_11,bids_limit_quantity_11,bids_limit_notional_11,asks_distance_11,asks_price_11,asks_limit_quantity_11,asks_limit_notional_11,bids_distance_12,bids_price_12,bids_limit_quantity_12,bids_limit_notional_12,asks_distance_12,asks_price_12,asks_limit_quantity_12,asks_limit_notional_12,bids_distance_13,bids_price_13,bids_limit_quantity_13,bids_limit_notional_13,asks_distance_13,asks_price_13,asks_limit_quantity_13,asks_limit_notional_13,bids_distance_14,bids_price_14,bids_limit_quantity_14,bids_limit_notional_14,asks_distance_14,asks_price_14,asks_limit_quantity_14,asks_limit_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:33:49.861733+00:00,1965.845,0.01,875154.482918,1.684774e+06,-0.000003,1965.84,44.10913,86711.492188,0.000003,1965.85,10.17655,20005.570312,-0.000155,1965.54,4.90975,9650.30957,0.000135,1966.11,1.645,3234.25,-0.00016,1965.53,15.099998,29679.5,0.00014,1966.12,10.17502,20005.310547,-0.000257,1965.34,5.074002,9972.139648,0.000196,1966.23,2.212371,4350.029785,-0.000262,1965.33,42.706839,83933.03125,0.000242,1966.32,2.97,5839.970215,-0.000277,1965.3,17.771969,34927.25,0.000277,1966.39,10.173689,20005.439453,-0.000425,1965.01,9.933492,19519.410156,0.000415,1966.66,1.97,3874.320068,-0.00043,1965.0,7.15,14049.75,0.000455,1966.74,0.372103,731.830017,-0.000455,1964.95,0.261788,514.400024,0.000511,1966.85,0.189999,373.700012,-0.000506,1964.85,0.382324,751.210022,0.000567,1966.96,0.100002,196.699997,-0.000532,1964.8,1.04715,2057.439941,0.000572,1966.97,1.047149,2059.709961,-0.000547,1964.77,0.387201,760.76001,0.000588,1967.0,1.97,3874.98999,-0.000593,1964.68,2.521576,4954.089844,0.000623,1967.07,3.0,5901.209961,-0.000613,1964.64,2.542634,4995.359863,0.000628,1967.08,0.090637,178.289993,-0.000674,1964.52,1.017292,1998.48999,0.000633,1967.09,14.494589,28512.160156



Debug ADA_1min dataframe after adding the quantity feature column:


(17109, 124)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_quantity_0,bids_limit_notional_0,asks_distance_0,asks_price_0,asks_limit_quantity_0,asks_limit_notional_0,bids_distance_1,bids_price_1,bids_limit_quantity_1,bids_limit_notional_1,asks_distance_1,asks_price_1,asks_limit_quantity_1,asks_limit_notional_1,bids_distance_2,bids_price_2,bids_limit_quantity_2,bids_limit_notional_2,asks_distance_2,asks_price_2,asks_limit_quantity_2,asks_limit_notional_2,bids_distance_3,bids_price_3,bids_limit_quantity_3,bids_limit_notional_3,asks_distance_3,asks_price_3,asks_limit_quantity_3,asks_limit_notional_3,bids_distance_4,bids_price_4,bids_limit_quantity_4,bids_limit_notional_4,asks_distance_4,asks_price_4,asks_limit_quantity_4,asks_limit_notional_4,bids_distance_5,bids_price_5,bids_limit_quantity_5,bids_limit_notional_5,asks_distance_5,asks_price_5,asks_limit_quantity_5,asks_limit_notional_5,bids_distance_6,bids_price_6,bids_limit_quantity_6,bids_limit_notional_6,asks_distance_6,asks_price_6,asks_limit_quantity_6,asks_limit_notional_6,bids_distance_7,bids_price_7,bids_limit_quantity_7,bids_limit_notional_7,asks_distance_7,asks_price_7,asks_limit_quantity_7,asks_limit_notional_7,bids_distance_8,bids_price_8,bids_limit_quantity_8,bids_limit_notional_8,asks_distance_8,asks_price_8,asks_limit_quantity_8,asks_limit_notional_8,bids_distance_9,bids_price_9,bids_limit_quantity_9,bids_limit_notional_9,asks_distance_9,asks_price_9,asks_limit_quantity_9,asks_limit_notional_9,bids_distance_10,bids_price_10,bids_limit_quantity_10,bids_limit_notional_10,asks_distance_10,asks_price_10,asks_limit_quantity_10,asks_limit_notional_10,bids_distance_11,bids_price_11,bids_limit_quantity_11,bids_limit_notional_11,asks_distance_11,asks_price_11,asks_limit_quantity_11,asks_limit_notional_11,bids_distance_12,bids_price_12,bids_limit_quantity_12,bids_limit_notional_12,asks_distance_12,asks_price_12,asks_limit_quantity_12,asks_limit_notional_12,bids_distance_13,bids_price_13,bids_limit_quantity_13,bids_limit_notional_13,asks_distance_13,asks_price_13,asks_limit_quantity_13,asks_limit_notional_13,bids_distance_14,bids_price_14,bids_limit_quantity_14,bids_limit_notional_14,asks_distance_14,asks_price_14,asks_limit_quantity_14,asks_limit_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:33:59.055697+00:00,1.16205,0.0001,56936.467913,258248.957367,-0.000043,1.162,210273.2358,244337.5,0.000043,1.1621,3415.816212,3969.52002,-0.000129,1.1619,25450.96024,29571.470703,0.000904,1.1631,3960.828685,4606.839844,-0.000301,1.1617,24936.498638,28968.730469,0.001162,1.1634,36015.000483,41899.851562,-0.000473,1.1615,16928.678567,19662.660156,0.001248,1.1635,528.001734,614.330017,-0.000731,1.1612,32061.539248,37229.859375,0.001678,1.164,6182.302574,7196.200195,-0.000818,1.1611,181485.35064,210722.640625,0.002108,1.1645,1700.000021,1979.650024,-0.000904,1.161,65389.373384,75917.0625,0.002194,1.1646,4658.818613,5425.660156,-0.00099,1.1609,39628.322448,46004.519531,0.00228,1.1647,9449.849947,11006.240234,-0.001162,1.1607,12431.713823,14429.490234,0.002367,1.1648,6804.223952,7925.560059,-0.001248,1.1606,111264.465846,129133.539062,0.002453,1.1649,8661.550213,10089.839844,-0.001334,1.1605,21239.302832,24648.210938,0.003227,1.1658,21373.965975,24917.769531,-0.00142,1.1604,108850.531072,126310.15625,0.004174,1.1669,856.971463,1000.0,-0.001506,1.1603,46590.122841,54058.519531,0.00426,1.167,1209.117428,1411.040039,-0.001592,1.1602,102950.89882,119443.632812,0.004776,1.1676,2905.310029,3392.23999,-0.001678,1.1601,120116.50396,139347.15625,0.005551,1.1685,374.993575,438.179993


In [16]:
# Add the cumulative quantity at each level
# Example. cum_quantity_0 = quantity_0
#          cum_quantity_1 = quantity_0 + quantity_1
#          cum_quantity_2 = quantity_0 + quantity_1 + quantity_2
#          ....

if (ADD_CUM_QUANTITY_FEATURE):
    for crypto_ob_key in crypto_ob_df_dict:
        for bid_ask_str in bid_ask_str_lst:
            for notional_str in notional_str_lst:
                quantity_level_df = pd.DataFrame()    # empty dataframe to hold all the values of a specific type of quantity (e.g. bids_limit_quantity)
                                                      # at each level (e.g. bids_limit_quantity_0, bids_limit_quantity_1, bids_limit_quantity_2, ....)                
                for level_num in range(NUM_ORDER_BOOK_LEVELS):
                    quantity_level_str = bid_ask_str + notional_str + "quantity" + "_" + str(level_num)
                    quantity_level_df[quantity_level_str] = crypto_ob_df_dict[crypto_ob_key][quantity_level_str]
                    
                    # Compute the cumulative quantity across columns (axis=1) of the same row 
                    cum_quantity_level_df = quantity_level_df.cumsum(axis = 1)
                    
                    # Copy the the cum quantity data frame to the main crypto data frame
                    # Want to insert the cum quantity column at a specific level right next to the quantity column for easy viewing (e.g. "bids_quantity_0", "bids_cum_quantity_0", ...)
                    quantity_column_index = crypto_ob_df_dict[crypto_ob_key].columns.get_loc(quantity_level_str)
                    cum_quantity_level_str = bid_ask_str + notional_str + "cum_quantity" + "_" + str(level_num)
                    crypto_ob_df_dict[crypto_ob_key].insert(quantity_column_index+1, cum_quantity_level_str, cum_quantity_level_df[quantity_level_str])
                    #print(f"{quantity_level_str}, {cum_quantity_level_str}")
                    
        if (TURN_ON_PRINT_DEBUG):
            print(f"\nDebug {crypto_ob_key} dataframe after adding cum quantity level columns:")
            display(crypto_ob_df_dict[crypto_ob_key].shape)
            display(crypto_ob_df_dict[crypto_ob_key].head(n=1))                                    


Debug BTC_5min dataframe after adding cum quantity level columns:


(3367, 154)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_quantity_0,bids_limit_cum_quantity_0,bids_limit_notional_0,asks_distance_0,asks_price_0,asks_limit_quantity_0,asks_limit_cum_quantity_0,asks_limit_notional_0,bids_distance_1,bids_price_1,bids_limit_quantity_1,bids_limit_cum_quantity_1,bids_limit_notional_1,asks_distance_1,asks_price_1,asks_limit_quantity_1,asks_limit_cum_quantity_1,asks_limit_notional_1,bids_distance_2,bids_price_2,bids_limit_quantity_2,bids_limit_cum_quantity_2,bids_limit_notional_2,asks_distance_2,asks_price_2,asks_limit_quantity_2,asks_limit_cum_quantity_2,asks_limit_notional_2,bids_distance_3,bids_price_3,bids_limit_quantity_3,bids_limit_cum_quantity_3,bids_limit_notional_3,asks_distance_3,asks_price_3,asks_limit_quantity_3,asks_limit_cum_quantity_3,asks_limit_notional_3,bids_distance_4,bids_price_4,bids_limit_quantity_4,bids_limit_cum_quantity_4,bids_limit_notional_4,asks_distance_4,asks_price_4,asks_limit_quantity_4,asks_limit_cum_quantity_4,asks_limit_notional_4,bids_distance_5,bids_price_5,bids_limit_quantity_5,bids_limit_cum_quantity_5,bids_limit_notional_5,asks_distance_5,asks_price_5,asks_limit_quantity_5,asks_limit_cum_quantity_5,asks_limit_notional_5,bids_distance_6,bids_price_6,bids_limit_quantity_6,bids_limit_cum_quantity_6,bids_limit_notional_6,asks_distance_6,asks_price_6,asks_limit_quantity_6,asks_limit_cum_quantity_6,asks_limit_notional_6,bids_distance_7,bids_price_7,bids_limit_quantity_7,bids_limit_cum_quantity_7,bids_limit_notional_7,asks_distance_7,asks_price_7,asks_limit_quantity_7,asks_limit_cum_quantity_7,asks_limit_notional_7,bids_distance_8,bids_price_8,bids_limit_quantity_8,bids_limit_cum_quantity_8,bids_limit_notional_8,asks_distance_8,asks_price_8,asks_limit_quantity_8,asks_limit_cum_quantity_8,asks_limit_notional_8,bids_distance_9,bids_price_9,bids_limit_quantity_9,bids_limit_cum_quantity_9,bids_limit_notional_9,asks_distance_9,asks_price_9,asks_limit_quantity_9,asks_limit_cum_quantity_9,asks_limit_notional_9,bids_distance_10,bids_price_10,bids_limit_quantity_10,bids_limit_cum_quantity_10,bids_limit_notional_10,asks_distance_10,asks_price_10,asks_limit_quantity_10,asks_limit_cum_quantity_10,asks_limit_notional_10,bids_distance_11,bids_price_11,bids_limit_quantity_11,bids_limit_cum_quantity_11,bids_limit_notional_11,asks_distance_11,asks_price_11,asks_limit_quantity_11,asks_limit_cum_quantity_11,asks_limit_notional_11,bids_distance_12,bids_price_12,bids_limit_quantity_12,bids_limit_cum_quantity_12,bids_limit_notional_12,asks_distance_12,asks_price_12,asks_limit_quantity_12,asks_limit_cum_quantity_12,asks_limit_notional_12,bids_distance_13,bids_price_13,bids_limit_quantity_13,bids_limit_cum_quantity_13,bids_limit_notional_13,asks_distance_13,asks_price_13,asks_limit_quantity_13,asks_limit_cum_quantity_13,asks_limit_notional_13,bids_distance_14,bids_price_14,bids_limit_quantity_14,bids_limit_cum_quantity_14,bids_limit_notional_14,asks_distance_14,asks_price_14,asks_limit_quantity_14,asks_limit_cum_quantity_14,asks_limit_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:37:41.122161+00:00,55899.995,0.01,1.199304e+07,1.348616e+07,-8.944544e-08,55899.99,0.363289,0.363289,20307.839844,8.944544e-08,55900.0,39.504311,39.504311,2208291.0,-0.000129,55892.76,0.013348,0.376637,746.049988,0.000033,55901.84,0.006175,39.510487,345.220001,-0.00013,55892.75,0.02,0.396637,1117.859985,0.000278,55915.510001,0.001927,39.512414,107.75,-0.000143,55892.02,0.05,0.446637,2794.600098,0.000345,55919.3,0.48332,39.995734,27026.910156,-0.000201,55888.78,0.252293,0.698929,14100.320312,0.000364,55920.36,0.005076,40.000809,283.829987,-0.000201,55888.77,0.1,0.798929,5588.879883,0.000421,55923.54,0.06279,40.063599,3511.439941,-0.00024,55886.56,0.00335,0.802279,187.220001,0.0005,55927.95,0.038697,40.102296,2164.22998,-0.000271,55884.86,0.5,1.302279,27942.429688


Debug ETH_5min dataframe after adding cum quantity level columns:


(3365, 154)

...

(17110, 154)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_quantity_0,bids_limit_cum_quantity_0,bids_limit_notional_0,asks_distance_0,asks_price_0,asks_limit_quantity_0,asks_limit_cum_quantity_0,asks_limit_notional_0,bids_distance_1,bids_price_1,bids_limit_quantity_1,bids_limit_cum_quantity_1,bids_limit_notional_1,asks_distance_1,asks_price_1,asks_limit_quantity_1,asks_limit_cum_quantity_1,asks_limit_notional_1,bids_distance_2,bids_price_2,bids_limit_quantity_2,bids_limit_cum_quantity_2,bids_limit_notional_2,asks_distance_2,asks_price_2,asks_limit_quantity_2,asks_limit_cum_quantity_2,asks_limit_notional_2,bids_distance_3,bids_price_3,bids_limit_quantity_3,bids_limit_cum_quantity_3,bids_limit_notional_3,asks_distance_3,asks_price_3,asks_limit_quantity_3,asks_limit_cum_quantity_3,asks_limit_notional_3,bids_distance_4,bids_price_4,bids_limit_quantity_4,bids_limit_cum_quantity_4,bids_limit_notional_4,asks_distance_4,asks_price_4,asks_limit_quantity_4,asks_limit_cum_quantity_4,asks_limit_notional_4,bids_distance_5,bids_price_5,bids_limit_quantity_5,bids_limit_cum_quantity_5,bids_limit_notional_5,asks_distance_5,asks_price_5,asks_limit_quantity_5,asks_limit_cum_quantity_5,asks_limit_notional_5,bids_distance_6,bids_price_6,bids_limit_quantity_6,bids_limit_cum_quantity_6,bids_limit_notional_6,asks_distance_6,asks_price_6,asks_limit_quantity_6,asks_limit_cum_quantity_6,asks_limit_notional_6,bids_distance_7,bids_price_7,bids_limit_quantity_7,bids_limit_cum_quantity_7,bids_limit_notional_7,asks_distance_7,asks_price_7,asks_limit_quantity_7,asks_limit_cum_quantity_7,asks_limit_notional_7,bids_distance_8,bids_price_8,bids_limit_quantity_8,bids_limit_cum_quantity_8,bids_limit_notional_8,asks_distance_8,asks_price_8,asks_limit_quantity_8,asks_limit_cum_quantity_8,asks_limit_notional_8,bids_distance_9,bids_price_9,bids_limit_quantity_9,bids_limit_cum_quantity_9,bids_limit_notional_9,asks_distance_9,asks_price_9,asks_limit_quantity_9,asks_limit_cum_quantity_9,asks_limit_notional_9,bids_distance_10,bids_price_10,bids_limit_quantity_10,bids_limit_cum_quantity_10,bids_limit_notional_10,asks_distance_10,asks_price_10,asks_limit_quantity_10,asks_limit_cum_quantity_10,asks_limit_notional_10,bids_distance_11,bids_price_11,bids_limit_quantity_11,bids_limit_cum_quantity_11,bids_limit_notional_11,asks_distance_11,asks_price_11,asks_limit_quantity_11,asks_limit_cum_quantity_11,asks_limit_notional_11,bids_distance_12,bids_price_12,bids_limit_quantity_12,bids_limit_cum_quantity_12,bids_limit_notional_12,asks_distance_12,asks_price_12,asks_limit_quantity_12,asks_limit_cum_quantity_12,asks_limit_notional_12,bids_distance_13,bids_price_13,bids_limit_quantity_13,bids_limit_cum_quantity_13,bids_limit_notional_13,asks_distance_13,asks_price_13,asks_limit_quantity_13,asks_limit_cum_quantity_13,asks_limit_notional_13,bids_distance_14,bids_price_14,bids_limit_quantity_14,bids_limit_cum_quantity_14,bids_limit_notional_14,asks_distance_14,asks_price_14,asks_limit_quantity_14,asks_limit_cum_quantity_14,asks_limit_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:33:49.861733+00:00,1965.845,0.01,875154.482918,1.684774e+06,-0.000003,1965.84,44.10913,44.10913,86711.492188,0.000003,1965.85,10.17655,10.17655,20005.570312,-0.000155,1965.54,4.90975,49.01888,9650.30957,0.000135,1966.11,1.645,11.821549,3234.25,-0.00016,1965.53,15.099998,64.118878,29679.5,0.00014,1966.12,10.17502,21.996569,20005.310547,-0.000257,1965.34,5.074002,69.192881,9972.139648,0.000196,1966.23,2.212371,24.20894,4350.029785,-0.000262,1965.33,42.706839,111.899719,83933.03125,0.000242,1966.32,2.97,27.17894,5839.970215,-0.000277,1965.3,17.771969,129.671688,34927.25,0.000277,1966.39,10.173689,37.352629,20005.439453,-0.000425,1965.01,9.933492,139.605179,19519.410156,0.000415,1966.66,1.97,39.322629,3874.320068,-0.00043,1965.0,7.15,146.755179,14049.75,0.000455,196


Debug ADA_1min dataframe after adding cum quantity level columns:


(17109, 154)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_quantity_0,bids_limit_cum_quantity_0,bids_limit_notional_0,asks_distance_0,asks_price_0,asks_limit_quantity_0,asks_limit_cum_quantity_0,asks_limit_notional_0,bids_distance_1,bids_price_1,bids_limit_quantity_1,bids_limit_cum_quantity_1,bids_limit_notional_1,asks_distance_1,asks_price_1,asks_limit_quantity_1,asks_limit_cum_quantity_1,asks_limit_notional_1,bids_distance_2,bids_price_2,bids_limit_quantity_2,bids_limit_cum_quantity_2,bids_limit_notional_2,asks_distance_2,asks_price_2,asks_limit_quantity_2,asks_limit_cum_quantity_2,asks_limit_notional_2,bids_distance_3,bids_price_3,bids_limit_quantity_3,bids_limit_cum_quantity_3,bids_limit_notional_3,asks_distance_3,asks_price_3,asks_limit_quantity_3,asks_limit_cum_quantity_3,asks_limit_notional_3,bids_distance_4,bids_price_4,bids_limit_quantity_4,bids_limit_cum_quantity_4,bids_limit_notional_4,asks_distance_4,asks_price_4,asks_limit_quantity_4,asks_limit_cum_quantity_4,asks_limit_notional_4,bids_distance_5,bids_price_5,bids_limit_quantity_5,bids_limit_cum_quantity_5,bids_limit_notional_5,asks_distance_5,asks_price_5,asks_limit_quantity_5,asks_limit_cum_quantity_5,asks_limit_notional_5,bids_distance_6,bids_price_6,bids_limit_quantity_6,bids_limit_cum_quantity_6,bids_limit_notional_6,asks_distance_6,asks_price_6,asks_limit_quantity_6,asks_limit_cum_quantity_6,asks_limit_notional_6,bids_distance_7,bids_price_7,bids_limit_quantity_7,bids_limit_cum_quantity_7,bids_limit_notional_7,asks_distance_7,asks_price_7,asks_limit_quantity_7,asks_limit_cum_quantity_7,asks_limit_notional_7,bids_distance_8,bids_price_8,bids_limit_quantity_8,bids_limit_cum_quantity_8,bids_limit_notional_8,asks_distance_8,asks_price_8,asks_limit_quantity_8,asks_limit_cum_quantity_8,asks_limit_notional_8,bids_distance_9,bids_price_9,bids_limit_quantity_9,bids_limit_cum_quantity_9,bids_limit_notional_9,asks_distance_9,asks_price_9,asks_limit_quantity_9,asks_limit_cum_quantity_9,asks_limit_notional_9,bids_distance_10,bids_price_10,bids_limit_quantity_10,bids_limit_cum_quantity_10,bids_limit_notional_10,asks_distance_10,asks_price_10,asks_limit_quantity_10,asks_limit_cum_quantity_10,asks_limit_notional_10,bids_distance_11,bids_price_11,bids_limit_quantity_11,bids_limit_cum_quantity_11,bids_limit_notional_11,asks_distance_11,asks_price_11,asks_limit_quantity_11,asks_limit_cum_quantity_11,asks_limit_notional_11,bids_distance_12,bids_price_12,bids_limit_quantity_12,bids_limit_cum_quantity_12,bids_limit_notional_12,asks_distance_12,asks_price_12,asks_limit_quantity_12,asks_limit_cum_quantity_12,asks_limit_notional_12,bids_distance_13,bids_price_13,bids_limit_quantity_13,bids_limit_cum_quantity_13,bids_limit_notional_13,asks_distance_13,asks_price_13,asks_limit_quantity_13,asks_limit_cum_quantity_13,asks_limit_notional_13,bids_distance_14,bids_price_14,bids_limit_quantity_14,bids_limit_cum_quantity_14,bids_limit_notional_14,asks_distance_14,asks_price_14,asks_limit_quantity_14,asks_limit_cum_quantity_14,asks_limit_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-04-07 11:33:59.055697+00:00,1.16205,0.0001,56936.467913,258248.957367,-0.000043,1.162,210273.2358,210273.2358,244337.5,0.000043,1.1621,3415.816212,3415.816212,3969.52002,-0.000129,1.1619,25450.96024,235724.196041,29571.470703,0.000904,1.1631,3960.828685,7376.644897,4606.839844,-0.000301,1.1617,24936.498638,260660.694679,28968.730469,0.001162,1.1634,36015.000483,43391.64538,41899.851562,-0.000473,1.1615,16928.678567,277589.373246,19662.660156,0.001248,1.1635,528.001734,43919.647113,614.330017,-0.000731,1.1612,32061.539248,309650.912494,37229.859375,0.001678,1.164,6182.302574,50101.949687,7196.200195,-0.000818,1.1611,181485.35064,491136.263134,210722.640625,0.002108,1.1645,1700.000021,51801.949708,1979.650024,-0.000904,1.161,65389.373384,556525.636519,75917.0625,0.002194,1

In [17]:
# Add the cumulative notional at each level
# Example. cum_notional_0 = notional_0
#          cum_notional_1 = notional_0 + notional_1
#          cum_notional_2 = notional_0 + notional_1 + notional_2
#          ....

if (ADD_CUM_NOTIONAL_FEATURE):
    for crypto_ob_key in crypto_ob_df_dict:
        for bid_ask_str in bid_ask_str_lst:
            for notional_str in notional_str_lst:
                notional_level_df = pd.DataFrame()    # empty dataframe to hold all the values of a specific type of notional (e.g. bids_limit_notional)
                                                      # at each level (e.g. bids_limit_notional_0, bids_limit_notional_1, bids_limit_notional_2, ....)                
                for level_num in range(NUM_ORDER_BOOK_LEVELS):
                    notional_level_str = bid_ask_str + notional_str + "notional" + "_" + str(level_num)
                    notional_level_df[notional_level_str] = crypto_ob_df_dict[crypto_ob_key][notional_level_str]
                    
                    # Compute the cumulative notional across columns (axis=1) of the same row 
                    cum_notional_level_df = notional_level_df.cumsum(axis = 1)
                    
                    # Copy the the cum notional data frame to the main crypto data frame
                    # Want to insert the cum notional column at a specific level right next to the notional column for easy viewing (e.g. "bids_notional_0", "bids_cum_notional_0", ...)
                    notional_column_index = crypto_ob_df_dict[crypto_ob_key].columns.get_loc(notional_level_str)
                    cum_notional_level_str = bid_ask_str + notional_str + "cum_notional" + "_" + str(level_num)
                    crypto_ob_df_dict[crypto_ob_key].insert(notional_column_index+1, cum_notional_level_str, cum_notional_level_df[notional_level_str])
                    #print(f"{notional_level_str}, {cum_notional_level_str}")
                    
        if (TURN_ON_PRINT_DEBUG):
            print(f"\nDebug {crypto_ob_key} dataframe after adding cum notional level columns:")
            display(crypto_ob_df_dict[crypto_ob_key].shape)
            display(crypto_ob_df_dict[crypto_ob_key].head(n=1)) 


Debug BTC_5min dataframe after adding cum notional level columns:


(3367, 184)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_quantity_0,bids_limit_cum_quantity_0,bids_limit_notional_0,bids_limit_cum_notional_0,asks_distance_0,asks_price_0,asks_limit_quantity_0,asks_limit_cum_quantity_0,asks_limit_notional_0,asks_limit_cum_notional_0,bids_distance_1,bids_price_1,bids_limit_quantity_1,bids_limit_cum_quantity_1,bids_limit_notional_1,bids_limit_cum_notional_1,asks_distance_1,asks_price_1,asks_limit_quantity_1,asks_limit_cum_quantity_1,asks_limit_notional_1,asks_limit_cum_notional_1,bids_distance_2,bids_price_2,bids_limit_quantity_2,bids_limit_cum_quantity_2,bids_limit_notional_2,bids_limit_cum_notional_2,asks_distance_2,asks_price_2,asks_limit_quantity_2,asks_limit_cum_quantity_2,asks_limit_notional_2,asks_limit_cum_notional_2,bids_distance_3,bids_price_3,bids_limit_quantity_3,bids_limit_cum_quantity_3,bids_limit_notional_3,bids_limit_cum_notional_3,asks_distance_3,asks_price_3,asks_limit_quantity_3,asks_limit_cum_quantity_3,asks_limit_notional_3,asks_limit_cum_notional_3,bids_distance_4,bids_price_4,bids_limit_quantity_4,bids_limit_cum_quantity_4,bids_limit_notional_4,bids_limit_cum_notional_4,asks_distance_4,asks_price_4,asks_limit_quantity_4,asks_limit_cum_quantity_4,asks_limit_notional_4,asks_limit_cum_notional_4,bids_distance_5,bids_price_5,bids_limit_quantity_5,bids_limit_cum_quantity_5,bids_limit_notional_5,bids_limit_cum_notional_5,asks_distance_5,asks_price_5,asks_limit_quantity_5,asks_limit_cum_quantity_5,asks_limit_notional_5,asks_limit_cum_notional_5,bids_distance_6,bids_price_6,bids_limit_quantity_6,bids_limit_cum_quantity_6,bids_limit_notional_6,bids_limit_cum_notional_6,asks_distance_6,asks_price_6,asks_limit_quantity_6,asks_limit_cum_quantity_6,asks_limit_notional_6,asks_limit_cum_notional_6,bids_distance_7,bids_price_7,bids_limit_quantity_7,bids_limit_cum_quantity_7,bids_limit_notional_7,bids_limit_cum_notional_7,asks_distance_7,asks_price_7,asks_limit_quantity_7,asks_limit_cum_quantity_7,asks_limit_notional_7,asks_limit_cum_notional_7,bids_distance_8,bids_price_8,bids_limit_quantity_8,bids_limit_cum_quantity_8,bids_limit_notional_8,bids_limit_cum_notional_8,asks_distance_8,asks_price_8,asks_limit_quantity_8,asks_limit_cum_quantity_8,asks_limit_notional_8,asks_limit_cum_notional_8,bids_distance_9,bids_price_9,bids_limit_quantity_9,bids_limit_cum_quantity_9,bids_limit_notional_9,bids_limit_cum_notional_9,asks_distance_9,asks_price_9,asks_limit_quantity_9,asks_limit_cum_quantity_9,asks_limit_notional_9,asks_limit_cum_notional_9,bids_distance_10,bids_price_10,bids_limit_quantity_10,bids_limit_cum_quantity_10,bids_limit_notional_10,bids_limit_cum_notional_10,asks_distance_10,asks_price_10,asks_limit_quantity_10,asks_limit_cum_quantity_10,asks_limit_notional_10,asks_limit_cum_notional_10,bids_distance_11,bids_price_11,bids_limit_quantity_11,bids_limit_cum_quantity_11,bids_limit_notional_11,bids_limit_cum_notional_11,asks_distance_11,asks_price_11,asks_limit_quantity_11,asks_limit_cum_quantity_11,asks_limit_notional_11,asks_limit_cum_notional_11,bids_distance_12,bids_price_12,bids_limit_quantity_12,bids_limit_cum_quantity_12,bids_limit_notional_12,bids_limit_cum_notional_12,asks_distance_12,asks_price_12,asks_limit_quantity_12,asks_limit_cum_quantity_12,asks_limit_notional_12,asks_limit_cum_notional_12,bids_distance_13,bids_price_13,bids_limit_quantity_13,bids_limit_cum_quantity_13,bids_limit_notional_13,bids_limit_cum_notional_13,asks_distance_13,asks_price_13,asks_limit_quantity_13,asks_limit_cum_quantity_13,asks_limit_notional_13,asks_limit_cum_notional_13,bids_distance_14,bids_price_14,bids_limit_quantity_14,bids_limit_cum_quantity_14,bids_limit_notional_14,bids_limit_cum_notional_14,asks_distance_14,asks_price_14,asks_limit_quantity_14,asks_limit_cum_quantity_14,asks_limit_notional_14,asks_limit_cum_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


Debug ETH_5min dataframe after adding cum notional level columns:


(3365, 184)

...

(17110, 184)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_quantity_0,bids_limit_cum_quantity_0,bids_limit_notional_0,bids_limit_cum_notional_0,asks_distance_0,asks_price_0,asks_limit_quantity_0,asks_limit_cum_quantity_0,asks_limit_notional_0,asks_limit_cum_notional_0,bids_distance_1,bids_price_1,bids_limit_quantity_1,bids_limit_cum_quantity_1,bids_limit_notional_1,bids_limit_cum_notional_1,asks_distance_1,asks_price_1,asks_limit_quantity_1,asks_limit_cum_quantity_1,asks_limit_notional_1,asks_limit_cum_notional_1,bids_distance_2,bids_price_2,bids_limit_quantity_2,bids_limit_cum_quantity_2,bids_limit_notional_2,bids_limit_cum_notional_2,asks_distance_2,asks_price_2,asks_limit_quantity_2,asks_limit_cum_quantity_2,asks_limit_notional_2,asks_limit_cum_notional_2,bids_distance_3,bids_price_3,bids_limit_quantity_3,bids_limit_cum_quantity_3,bids_limit_notional_3,bids_limit_cum_notional_3,asks_distance_3,asks_price_3,asks_limit_quantity_3,asks_limit_cum_quantity_3,asks_limit_notional_3,asks_limit_cum_notional_3,bids_distance_4,bids_price_4,bids_limit_quantity_4,bids_limit_cum_quantity_4,bids_limit_notional_4,bids_limit_cum_notional_4,asks_distance_4,asks_price_4,asks_limit_quantity_4,asks_limit_cum_quantity_4,asks_limit_notional_4,asks_limit_cum_notional_4,bids_distance_5,bids_price_5,bids_limit_quantity_5,bids_limit_cum_quantity_5,bids_limit_notional_5,bids_limit_cum_notional_5,asks_distance_5,asks_price_5,asks_limit_quantity_5,asks_limit_cum_quantity_5,asks_limit_notional_5,asks_limit_cum_notional_5,bids_distance_6,bids_price_6,bids_limit_quantity_6,bids_limit_cum_quantity_6,bids_limit_notional_6,bids_limit_cum_notional_6,asks_distance_6,asks_price_6,asks_limit_quantity_6,asks_limit_cum_quantity_6,asks_limit_notional_6,asks_limit_cum_notional_6,bids_distance_7,bids_price_7,bids_limit_quantity_7,bids_limit_cum_quantity_7,bids_limit_notional_7,bids_limit_cum_notional_7,asks_distance_7,asks_price_7,asks_limit_quantity_7,asks_limit_cum_quantity_7,asks_limit_notional_7,asks_limit_cum_notional_7,bids_distance_8,bids_price_8,bids_limit_quantity_8,bids_limit_cum_quantity_8,bids_limit_notional_8,bids_limit_cum_notional_8,asks_distance_8,asks_price_8,asks_limit_quantity_8,asks_limit_cum_quantity_8,asks_limit_notional_8,asks_limit_cum_notional_8,bids_distance_9,bids_price_9,bids_limit_quantity_9,bids_limit_cum_quantity_9,bids_limit_notional_9,bids_limit_cum_notional_9,asks_distance_9,asks_price_9,asks_limit_quantity_9,asks_limit_cum_quantity_9,asks_limit_notional_9,asks_limit_cum_notional_9,bids_distance_10,bids_price_10,bids_limit_quantity_10,bids_limit_cum_quantity_10,bids_limit_notional_10,bids_limit_cum_notional_10,asks_distance_10,asks_price_10,asks_limit_quantity_10,asks_limit_cum_quantity_10,asks_limit_notional_10,asks_limit_cum_notional_10,bids_distance_11,bids_price_11,bids_limit_quantity_11,bids_limit_cum_quantity_11,bids_limit_notional_11,bids_limit_cum_notional_11,asks_distance_11,asks_price_11,asks_limit_quantity_11,asks_limit_cum_quantity_11,asks_limit_notional_11,asks_limit_cum_notional_11,bids_distance_12,bids_price_12,bids_limit_quantity_12,bids_limit_cum_quantity_12,bids_limit_notional_12,bids_limit_cum_notional_12,asks_distance_12,asks_price_12,asks_limit_quantity_12,asks_limit_cum_quantity_12,asks_limit_notional_12,asks_limit_cum_notional_12,bids_distance_13,bids_price_13,bids_limit_quantity_13,bids_limit_cum_quantity_13,bids_limit_notional_13,bids_limit_cum_notional_13,asks_distance_13,asks_price_13,asks_limit_quantity_13,asks_limit_cum_quantity_13,asks_limit_notional_13,asks_limit_cum_notional_13,bids_distance_14,bids_price_14,bids_limit_quantity_14,bids_limit_cum_quantity_14,bids_limit_notional_14,bids_limit_cum_notional_14,asks_distance_14,asks_price_14,asks_limit_quantity_14,asks_limit_cum_quantity_14,asks_limit_notional_14,asks_limit_cum_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


Debug ADA_1min dataframe after adding cum notional level columns:


(17109, 184)

,midpoint,spread,buys,sells,bids_distance_0,bids_price_0,bids_limit_quantity_0,bids_limit_cum_quantity_0,bids_limit_notional_0,bids_limit_cum_notional_0,asks_distance_0,asks_price_0,asks_limit_quantity_0,asks_limit_cum_quantity_0,asks_limit_notional_0,asks_limit_cum_notional_0,bids_distance_1,bids_price_1,bids_limit_quantity_1,bids_limit_cum_quantity_1,bids_limit_notional_1,bids_limit_cum_notional_1,asks_distance_1,asks_price_1,asks_limit_quantity_1,asks_limit_cum_quantity_1,asks_limit_notional_1,asks_limit_cum_notional_1,bids_distance_2,bids_price_2,bids_limit_quantity_2,bids_limit_cum_quantity_2,bids_limit_notional_2,bids_limit_cum_notional_2,asks_distance_2,asks_price_2,asks_limit_quantity_2,asks_limit_cum_quantity_2,asks_limit_notional_2,asks_limit_cum_notional_2,bids_distance_3,bids_price_3,bids_limit_quantity_3,bids_limit_cum_quantity_3,bids_limit_notional_3,bids_limit_cum_notional_3,asks_distance_3,asks_price_3,asks_limit_quantity_3,asks_limit_cum_quantity_3,asks_limit_notional_3,asks_limit_cum_notional_3,bids_distance_4,bids_price_4,bids_limit_quantity_4,bids_limit_cum_quantity_4,bids_limit_notional_4,bids_limit_cum_notional_4,asks_distance_4,asks_price_4,asks_limit_quantity_4,asks_limit_cum_quantity_4,asks_limit_notional_4,asks_limit_cum_notional_4,bids_distance_5,bids_price_5,bids_limit_quantity_5,bids_limit_cum_quantity_5,bids_limit_notional_5,bids_limit_cum_notional_5,asks_distance_5,asks_price_5,asks_limit_quantity_5,asks_limit_cum_quantity_5,asks_limit_notional_5,asks_limit_cum_notional_5,bids_distance_6,bids_price_6,bids_limit_quantity_6,bids_limit_cum_quantity_6,bids_limit_notional_6,bids_limit_cum_notional_6,asks_distance_6,asks_price_6,asks_limit_quantity_6,asks_limit_cum_quantity_6,asks_limit_notional_6,asks_limit_cum_notional_6,bids_distance_7,bids_price_7,bids_limit_quantity_7,bids_limit_cum_quantity_7,bids_limit_notional_7,bids_limit_cum_notional_7,asks_distance_7,asks_price_7,asks_limit_quantity_7,asks_limit_cum_quantity_7,asks_limit_notional_7,asks_limit_cum_notional_7,bids_distance_8,bids_price_8,bids_limit_quantity_8,bids_limit_cum_quantity_8,bids_limit_notional_8,bids_limit_cum_notional_8,asks_distance_8,asks_price_8,asks_limit_quantity_8,asks_limit_cum_quantity_8,asks_limit_notional_8,asks_limit_cum_notional_8,bids_distance_9,bids_price_9,bids_limit_quantity_9,bids_limit_cum_quantity_9,bids_limit_notional_9,bids_limit_cum_notional_9,asks_distance_9,asks_price_9,asks_limit_quantity_9,asks_limit_cum_quantity_9,asks_limit_notional_9,asks_limit_cum_notional_9,bids_distance_10,bids_price_10,bids_limit_quantity_10,bids_limit_cum_quantity_10,bids_limit_notional_10,bids_limit_cum_notional_10,asks_distance_10,asks_price_10,asks_limit_quantity_10,asks_limit_cum_quantity_10,asks_limit_notional_10,asks_limit_cum_notional_10,bids_distance_11,bids_price_11,bids_limit_quantity_11,bids_limit_cum_quantity_11,bids_limit_notional_11,bids_limit_cum_notional_11,asks_distance_11,asks_price_11,asks_limit_quantity_11,asks_limit_cum_quantity_11,asks_limit_notional_11,asks_limit_cum_notional_11,bids_distance_12,bids_price_12,bids_limit_quantity_12,bids_limit_cum_quantity_12,bids_limit_notional_12,bids_limit_cum_notional_12,asks_distance_12,asks_price_12,asks_limit_quantity_12,asks_limit_cum_quantity_12,asks_limit_notional_12,asks_limit_cum_notional_12,bids_distance_13,bids_price_13,bids_limit_quantity_13,bids_limit_cum_quantity_13,bids_limit_notional_13,bids_limit_cum_notional_13,asks_distance_13,asks_price_13,asks_limit_quantity_13,asks_limit_cum_quantity_13,asks_limit_notional_13,asks_limit_cum_notional_13,bids_distance_14,bids_price_14,bids_limit_quantity_14,bids_limit_cum_quantity_14,bids_limit_notional_14,bids_limit_cum_notional_14,asks_distance_14,asks_price_14,asks_limit_quantity_14,asks_limit_cum_quantity_14,asks_limit_notional_14,asks_limit_cum_notional_14
system_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [18]:
# Add the bid-ask imbalance at each level
# Imbalance = bid_notional / (bid_notional + ask_notional)
#    < 0.5, more buyers than sellers
#    > 0.5, more sellers than buyers
#    How may this move the prices?

if (ADD_BID_ASK_IMBALANCE):
    for crypto_ob_key in crypto_ob_df_dict:
        for level_num in range(NUM_ORDER_BOOK_LEVELS):
            for notional_str in notional_str_lst:
                bid_notional_level_str = "bids" + notional_str + "notional" + "_" + str(level_num)
                ask_notional_level_str = "asks" + notional_str + "notional" + "_" + str(level_num)            
                imbalance_level_str = "bids_asks_imbalance" + notional_str + "notional" + "_" + str(level_num)            
            
                # bid-ask imbalance = bid notional / (bid notional + ask notional)
                bid_ask_imbalance = crypto_ob_df_dict[crypto_ob_key][bid_notional_level_str] / (crypto_ob_df_dict[crypto_ob_key][bid_notional_level_str] + crypto_ob_df_dict[crypto_ob_key][ask_notional_level_str])
                #a = bid_ask_imbalance.isnull().sum()
                # Replace any imbalance which resulted in Nan (due to divide by 0) with 0
                bid_ask_imbalance.fillna(0, inplace=True)                
                                                                                                
                # Want to insert the quantity column at a specific level right after the ask notional column for easy viewing
                notional_column_index = crypto_ob_df_dict[crypto_ob_key].columns.get_loc(ask_notional_level_str)
                crypto_ob_df_dict[crypto_ob_key].insert(notional_column_index+1, imbalance_level_str, bid_ask_imbalance)   
                #print(f"{bid_notional_level_str}, {ask_notional_level_str}, {imbalance_level_str}")
        
        crypto_ob_df_dict[crypto_ob_key].insert(crypto_ob_df_dict[crypto_ob_key].columns.get_loc("sells")+1, "buys_sells_imbalance", crypto_ob_df_dict[crypto_ob_key]['buys'] / (crypto_ob_df_dict[crypto_ob_key]['buys'] + crypto_ob_df_dict[crypto_ob_key]['sells'])) 

        # Replace any imbalance which resulted in Nan (due to divide by 0) with 0
        crypto_ob_df_dict[crypto_ob_key].replace(np.nan, 0.0)
        
        if (TURN_ON_PRINT_DEBUG):
            print(f"\nDebug {crypto_ob_key} dataframe after adding the bid-ask imbalance feature column:")
            display(crypto_ob_df_dict[crypto_ob_key].shape)
            display(crypto_ob_df_dict[crypto_ob_key].head(n=1))           


Debug BTC_5min dataframe after adding the bid-ask imbalance feature column:


(3367, 200)

,midpoint,spread,buys,sells,buys_sells_imbalance,bids_distance_0,bids_price_0,bids_limit_quantity_0,bids_limit_cum_quantity_0,bids_limit_notional_0,bids_limit_cum_notional_0,asks_distance_0,asks_price_0,asks_limit_quantity_0,asks_limit_cum_quantity_0,asks_limit_notional_0,bids_asks_imbalance_limit_notional_0,asks_limit_cum_notional_0,bids_distance_1,bids_price_1,bids_limit_quantity_1,bids_limit_cum_quantity_1,bids_limit_notional_1,bids_limit_cum_notional_1,asks_distance_1,asks_price_1,asks_limit_quantity_1,asks_limit_cum_quantity_1,asks_limit_notional_1,bids_asks_imbalance_limit_notional_1,asks_limit_cum_notional_1,bids_distance_2,bids_price_2,bids_limit_quantity_2,bids_limit_cum_quantity_2,bids_limit_notional_2,bids_limit_cum_notional_2,asks_distance_2,asks_price_2,asks_limit_quantity_2,asks_limit_cum_quantity_2,asks_limit_notional_2,bids_asks_imbalance_limit_notional_2,asks_limit_cum_notional_2,bids_distance_3,bids_price_3,bids_limit_quantity_3,bids_limit_cum_quantity_3,bids_limit_notional_3,bids_limit_cum_notional_3,asks_distance_3,asks_price_3,asks_limit_quantity_3,asks_limit_cum_quantity_3,asks_limit_notional_3,bids_asks_imbalance_limit_notional_3,asks_limit_cum_notional_3,bids_distance_4,bids_price_4,bids_limit_quantity_4,bids_limit_cum_quantity_4,bids_limit_notional_4,bids_limit_cum_notional_4,asks_distance_4,asks_price_4,asks_limit_quantity_4,asks_limit_cum_quantity_4,asks_limit_notional_4,bids_asks_imbalance_limit_notional_4,asks_limit_cum_notional_4,bids_distance_5,bids_price_5,bids_limit_quantity_5,bids_limit_cum_quantity_5,bids_limit_notional_5,bids_limit_cum_notional_5,asks_distance_5,asks_price_5,asks_limit_quantity_5,asks_limit_cum_quantity_5,asks_limit_notional_5,bids_asks_imbalance_limit_notional_5,asks_limit_cum_notional_5,bids_distance_6,bids_price_6,bids_limit_quantity_6,bids_limit_cum_quantity_6,bids_limit_notional_6,bids_limit_cum_notional_6,asks_distance_6,asks_price_6,asks_limit_quantity_6,asks_limit_cum_quantity_6,asks_limit_notional_6,bids_asks_imbalance_limit_notional_6,asks_limit_cum_notional_6,bids_distance_7,bids_price_7,bids_limit_quantity_7,bids_limit_cum_quantity_7,bids_limit_notional_7,bids_limit_cum_notional_7,asks_distance_7,asks_price_7,asks_limit_quantity_7,asks_limit_cum_quantity_7,asks_limit_notional_7,bids_asks_imbalance_limit_notional_7,asks_limit_cum_notional_7,bids_distance_8,bids_price_8,bids_limit_quantity_8,bids_limit_cum_quantity_8,bids_limit_notional_8,bids_limit_cum_notional_8,asks_distance_8,asks_price_8,asks_limit_quantity_8,asks_limit_cum_quantity_8,asks_limit_notional_8,bids_asks_imbalance_limit_notional_8,asks_limit_cum_notional_8,bids_distance_9,bids_price_9,bids_limit_quantity_9,bids_limit_cum_quantity_9,bids_limit_notional_9,bids_limit_cum_notional_9,asks_distance_9,asks_price_9,asks_limit_quantity_9,asks_limit_cum_quantity_9,asks_limit_notional_9,bids_asks_imbalance_limit_notional_9,asks_limit_cum_notional_9,bids_distance_10,bids_price_10,bids_limit_quantity_10,bids_limit_cum_quantity_10,bids_limit_notional_10,bids_limit_cum_notional_10,asks_distance_10,asks_price_10,asks_limit_quantity_10,asks_limit_cum_quantity_10,asks_limit_notional_10,bids_asks_imbalance_limit_notional_10,asks_limit_cum_notional_10,bids_distance_11,bids_price_11,bids_limit_quantity_11,bids_limit_cum_quantity_11,bids_limit_notional_11,bids_limit_cum_notional_11,asks_distance_11,asks_price_11,asks_limit_quantity_11,asks_limit_cum_quantity_11,asks_limit_notional_11,bids_asks_imbalance_limit_notional_11,asks_limit_cum_notional_11,bids_distance_12,bids_price_12,bids_limit_quantity_12,bids_limit_cum_quantity_12,bids_limit_notional_12,bids_limit_cum_notional_12,asks_distance_12,asks_price_12,asks_limit_quantity_12,asks_limit_cum_quantity_12,asks_limit_notional_12,bids_asks_imbalance_limit_notional_12,asks_limit_cum_notional_12,bids_distance_13,bids_price_13,bids_limit_quantity_13,bids_limit_cum_quantity_13,bids_limit_notional_13,bids_limit_cum_notional_13,asks_distance_13,asks_price_13,asks_limit_quantity


Debug ETH_5min dataframe after adding the bid-ask imbalance feature column:


(3365, 200)

...

(17110, 200)

,midpoint,spread,buys,sells,buys_sells_imbalance,bids_distance_0,bids_price_0,bids_limit_quantity_0,bids_limit_cum_quantity_0,bids_limit_notional_0,bids_limit_cum_notional_0,asks_distance_0,asks_price_0,asks_limit_quantity_0,asks_limit_cum_quantity_0,asks_limit_notional_0,bids_asks_imbalance_limit_notional_0,asks_limit_cum_notional_0,bids_distance_1,bids_price_1,bids_limit_quantity_1,bids_limit_cum_quantity_1,bids_limit_notional_1,bids_limit_cum_notional_1,asks_distance_1,asks_price_1,asks_limit_quantity_1,asks_limit_cum_quantity_1,asks_limit_notional_1,bids_asks_imbalance_limit_notional_1,asks_limit_cum_notional_1,bids_distance_2,bids_price_2,bids_limit_quantity_2,bids_limit_cum_quantity_2,bids_limit_notional_2,bids_limit_cum_notional_2,asks_distance_2,asks_price_2,asks_limit_quantity_2,asks_limit_cum_quantity_2,asks_limit_notional_2,bids_asks_imbalance_limit_notional_2,asks_limit_cum_notional_2,bids_distance_3,bids_price_3,bids_limit_quantity_3,bids_limit_cum_quantity_3,bids_limit_notional_3,bids_limit_cum_notional_3,asks_distance_3,asks_price_3,asks_limit_quantity_3,asks_limit_cum_quantity_3,asks_limit_notional_3,bids_asks_imbalance_limit_notional_3,asks_limit_cum_notional_3,bids_distance_4,bids_price_4,bids_limit_quantity_4,bids_limit_cum_quantity_4,bids_limit_notional_4,bids_limit_cum_notional_4,asks_distance_4,asks_price_4,asks_limit_quantity_4,asks_limit_cum_quantity_4,asks_limit_notional_4,bids_asks_imbalance_limit_notional_4,asks_limit_cum_notional_4,bids_distance_5,bids_price_5,bids_limit_quantity_5,bids_limit_cum_quantity_5,bids_limit_notional_5,bids_limit_cum_notional_5,asks_distance_5,asks_price_5,asks_limit_quantity_5,asks_limit_cum_quantity_5,asks_limit_notional_5,bids_asks_imbalance_limit_notional_5,asks_limit_cum_notional_5,bids_distance_6,bids_price_6,bids_limit_quantity_6,bids_limit_cum_quantity_6,bids_limit_notional_6,bids_limit_cum_notional_6,asks_distance_6,asks_price_6,asks_limit_quantity_6,asks_limit_cum_quantity_6,asks_limit_notional_6,bids_asks_imbalance_limit_notional_6,asks_limit_cum_notional_6,bids_distance_7,bids_price_7,bids_limit_quantity_7,bids_limit_cum_quantity_7,bids_limit_notional_7,bids_limit_cum_notional_7,asks_distance_7,asks_price_7,asks_limit_quantity_7,asks_limit_cum_quantity_7,asks_limit_notional_7,bids_asks_imbalance_limit_notional_7,asks_limit_cum_notional_7,bids_distance_8,bids_price_8,bids_limit_quantity_8,bids_limit_cum_quantity_8,bids_limit_notional_8,bids_limit_cum_notional_8,asks_distance_8,asks_price_8,asks_limit_quantity_8,asks_limit_cum_quantity_8,asks_limit_notional_8,bids_asks_imbalance_limit_notional_8,asks_limit_cum_notional_8,bids_distance_9,bids_price_9,bids_limit_quantity_9,bids_limit_cum_quantity_9,bids_limit_notional_9,bids_limit_cum_notional_9,asks_distance_9,asks_price_9,asks_limit_quantity_9,asks_limit_cum_quantity_9,asks_limit_notional_9,bids_asks_imbalance_limit_notional_9,asks_limit_cum_notional_9,bids_distance_10,bids_price_10,bids_limit_quantity_10,bids_limit_cum_quantity_10,bids_limit_notional_10,bids_limit_cum_notional_10,asks_distance_10,asks_price_10,asks_limit_quantity_10,asks_limit_cum_quantity_10,asks_limit_notional_10,bids_asks_imbalance_limit_notional_10,asks_limit_cum_notional_10,bids_distance_11,bids_price_11,bids_limit_quantity_11,bids_limit_cum_quantity_11,bids_limit_notional_11,bids_limit_cum_notional_11,asks_distance_11,asks_price_11,asks_limit_quantity_11,asks_limit_cum_quantity_11,asks_limit_notional_11,bids_asks_imbalance_limit_notional_11,asks_limit_cum_notional_11,bids_distance_12,bids_price_12,bids_limit_quantity_12,bids_limit_cum_quantity_12,bids_limit_notional_12,bids_limit_cum_notional_12,asks_distance_12,asks_price_12,asks_limit_quantity_12,asks_limit_cum_quantity_12,asks_limit_notional_12,bids_asks_imbalance_limit_notional_12,asks_limit_cum_notional_12,bids_distance_13,bids_price_13,bids_limit_quantity_13,bids_limit_cum_quantity_13,bids_limit_notional_13,bids_limit_cum_notional_13,asks_distance_13,asks_price_13,asks_limit_quantity


Debug ADA_1min dataframe after adding the bid-ask imbalance feature column:


(17109, 200)

,midpoint,spread,buys,sells,buys_sells_imbalance,bids_distance_0,bids_price_0,bids_limit_quantity_0,bids_limit_cum_quantity_0,bids_limit_notional_0,bids_limit_cum_notional_0,asks_distance_0,asks_price_0,asks_limit_quantity_0,asks_limit_cum_quantity_0,asks_limit_notional_0,bids_asks_imbalance_limit_notional_0,asks_limit_cum_notional_0,bids_distance_1,bids_price_1,bids_limit_quantity_1,bids_limit_cum_quantity_1,bids_limit_notional_1,bids_limit_cum_notional_1,asks_distance_1,asks_price_1,asks_limit_quantity_1,asks_limit_cum_quantity_1,asks_limit_notional_1,bids_asks_imbalance_limit_notional_1,asks_limit_cum_notional_1,bids_distance_2,bids_price_2,bids_limit_quantity_2,bids_limit_cum_quantity_2,bids_limit_notional_2,bids_limit_cum_notional_2,asks_distance_2,asks_price_2,asks_limit_quantity_2,asks_limit_cum_quantity_2,asks_limit_notional_2,bids_asks_imbalance_limit_notional_2,asks_limit_cum_notional_2,bids_distance_3,bids_price_3,bids_limit_quantity_3,bids_limit_cum_quantity_3,bids_limit_notional_3,bids_limit_cum_notional_3,asks_distance_3,asks_price_3,asks_limit_quantity_3,asks_limit_cum_quantity_3,asks_limit_notional_3,bids_asks_imbalance_limit_notional_3,asks_limit_cum_notional_3,bids_distance_4,bids_price_4,bids_limit_quantity_4,bids_limit_cum_quantity_4,bids_limit_notional_4,bids_limit_cum_notional_4,asks_distance_4,asks_price_4,asks_limit_quantity_4,asks_limit_cum_quantity_4,asks_limit_notional_4,bids_asks_imbalance_limit_notional_4,asks_limit_cum_notional_4,bids_distance_5,bids_price_5,bids_limit_quantity_5,bids_limit_cum_quantity_5,bids_limit_notional_5,bids_limit_cum_notional_5,asks_distance_5,asks_price_5,asks_limit_quantity_5,asks_limit_cum_quantity_5,asks_limit_notional_5,bids_asks_imbalance_limit_notional_5,asks_limit_cum_notional_5,bids_distance_6,bids_price_6,bids_limit_quantity_6,bids_limit_cum_quantity_6,bids_limit_notional_6,bids_limit_cum_notional_6,asks_distance_6,asks_price_6,asks_limit_quantity_6,asks_limit_cum_quantity_6,asks_limit_notional_6,bids_asks_imbalance_limit_notional_6,asks_limit_cum_notional_6,bids_distance_7,bids_price_7,bids_limit_quantity_7,bids_limit_cum_quantity_7,bids_limit_notional_7,bids_limit_cum_notional_7,asks_distance_7,asks_price_7,asks_limit_quantity_7,asks_limit_cum_quantity_7,asks_limit_notional_7,bids_asks_imbalance_limit_notional_7,asks_limit_cum_notional_7,bids_distance_8,bids_price_8,bids_limit_quantity_8,bids_limit_cum_quantity_8,bids_limit_notional_8,bids_limit_cum_notional_8,asks_distance_8,asks_price_8,asks_limit_quantity_8,asks_limit_cum_quantity_8,asks_limit_notional_8,bids_asks_imbalance_limit_notional_8,asks_limit_cum_notional_8,bids_distance_9,bids_price_9,bids_limit_quantity_9,bids_limit_cum_quantity_9,bids_limit_notional_9,bids_limit_cum_notional_9,asks_distance_9,asks_price_9,asks_limit_quantity_9,asks_limit_cum_quantity_9,asks_limit_notional_9,bids_asks_imbalance_limit_notional_9,asks_limit_cum_notional_9,bids_distance_10,bids_price_10,bids_limit_quantity_10,bids_limit_cum_quantity_10,bids_limit_notional_10,bids_limit_cum_notional_10,asks_distance_10,asks_price_10,asks_limit_quantity_10,asks_limit_cum_quantity_10,asks_limit_notional_10,bids_asks_imbalance_limit_notional_10,asks_limit_cum_notional_10,bids_distance_11,bids_price_11,bids_limit_quantity_11,bids_limit_cum_quantity_11,bids_limit_notional_11,bids_limit_cum_notional_11,asks_distance_11,asks_price_11,asks_limit_quantity_11,asks_limit_cum_quantity_11,asks_limit_notional_11,bids_asks_imbalance_limit_notional_11,asks_limit_cum_notional_11,bids_distance_12,bids_price_12,bids_limit_quantity_12,bids_limit_cum_quantity_12,bids_limit_notional_12,bids_limit_cum_notional_12,asks_distance_12,asks_price_12,asks_limit_quantity_12,asks_limit_cum_quantity_12,asks_limit_notional_12,bids_asks_imbalance_limit_notional_12,asks_limit_cum_notional_12,bids_distance_13,bids_price_13,bids_limit_quantity_13,bids_limit_cum_quantity_13,bids_limit_notional_13,bids_limit_cum_notional_13,asks_distance_13,asks_price_13,asks_limit_quantity

In [19]:
# Replace any imbalance which resulted in Nan (due to divide by 0) with 0
for crypto_ob_key in crypto_ob_df_dict:
    crypto_ob_df_dict[crypto_ob_key].fillna(0, inplace=True)

In [20]:
# Save resultant crypto dataframe to be used in machine learning models to csv
if (WRITE_TO_CSC_FILE):
    for crypto_ob_key in crypto_ob_df_dict:
        csv_file = "./derived_data/crypto_" + crypto_ob_key + ".csv"
        crypto_ob_df_dict[crypto_ob_key].to_csv(csv_file, sep=',', index=True,header=True)